In [1]:
%pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import psycopg2
import pandas as pd

In [3]:
# Conexão com o banco de dados padrão
connection = psycopg2.connect(
    dbname="postgres", user="postgres", password="1234", host="localhost"
)
connection.autocommit = True
cursor = connection.cursor()

In [4]:
# Desabilitar restrições temporariamente
cursor.execute("SET session_replication_role = 'replica';")

# Obter todas as tabelas e excluir uma por uma
cursor.execute("""
    SELECT tablename 
    FROM pg_tables 
    WHERE schemaname = 'public';
""")
tables = cursor.fetchall()

for table in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table[0]} CASCADE;")
    print(f"Tabela {table[0]} deletada com sucesso!")

# Reabilitar restrições
cursor.execute("SET session_replication_role = 'origin';")

# Deleto o Banco de Dados se existir
cursor.execute(f"DROP DATABASE IF EXISTS pokemon_db;")
# Crio o Banco de Dados
cursor.execute(f"CREATE DATABASE pokemon_db;")

print("Banco de dados criado com sucesso!")

Tabela insignia deletada com sucesso!
Tabela ginasio deletada com sucesso!
Tabela tipo deletada com sucesso!
Tabela golpe deletada com sucesso!
Tabela natureza deletada com sucesso!
Tabela treinador deletada com sucesso!
Tabela pokemon deletada com sucesso!
Tabela item deletada com sucesso!
Tabela item_evolucao deletada com sucesso!
Tabela item_batalha deletada com sucesso!
Tabela item_pokebola deletada com sucesso!
Tabela item_medicinais deletada com sucesso!
Tabela equipe deletada com sucesso!
Tabela pode_aprender deletada com sucesso!
Tabela batalha deletada com sucesso!
Tabela pokedex deletada com sucesso!
Tabela pokemon_eh_de_tal deletada com sucesso!
Tabela item_eh_de_tal deletada com sucesso!
Tabela liga deletada com sucesso!
Tabela eh_formada deletada com sucesso!
Tabela eh_composta deletada com sucesso!
Tabela possui deletada com sucesso!
Tabela sabe deletada com sucesso!
Banco de dados criado com sucesso!


In [5]:
# Criação das tabelas no banco de dados
sql_commands = [
    """
    CREATE TABLE Insignia (
        id_insignia INTEGER PRIMARY KEY,
        nome VARCHAR(50) NOT NULL,
        descricao VARCHAR(100)
    );
    """,
    """
    CREATE TABLE Ginasio (
        id_ginasio INTEGER PRIMARY KEY,
        id_insignia INTEGER,
        nome VARCHAR(50) NOT NULL,
        cidade VARCHAR(50) NOT NULL,
        FOREIGN key (id_insignia) REFERENCES Insignia(id_insignia)
    );
    """,
    """
    CREATE TABLE Treinador (
        id_treinador INTEGER PRIMARY KEY,
        nome VARCHAR(50) NOT NULL,
        eh_campeao BOOLEAN

    );
    """,
    """
    CREATE TABLE Tipo (
        id_tipo SERIAL PRIMARY KEY,
        nome VARCHAR(50) NOT NULL
    );
    """,
    """
    CREATE TABLE Golpe (
        id_golpe SERIAL PRIMARY KEY,
        nome VARCHAR(50) NOT NULL,
        poder INTEGER,
        precisao INTEGER,
        id_tipo INTEGER,
        FOREIGN KEY (id_tipo) REFERENCES Tipo(id_tipo)
    );
    """,
    """
    CREATE TABLE Natureza (
        id_natureza SERIAL PRIMARY KEY,
        descricao VARCHAR(100) NOT NULL,
        efeito VARCHAR(100)
    );
    """,
    """
    CREATE TABLE Pokemon (
        id_pokedex_pokemon INTEGER,
        id_instancia_pokemon INTEGER,
        id_treinador INTEGER,
        nome VARCHAR(50) NOT NULL,
        nivel INTEGER,
        evolui_com_pedra BOOLEAN,
        condicao VARCHAR(20),
        hp INTEGER NOT NULL CHECK (hp >= 0),
        ataque INTEGER NOT NULL CHECK (ataque >= 0),
        defesa INTEGER NOT NULL CHECK (defesa >= 0),
        ataque_especial INTEGER NOT NULL CHECK (ataque_especial >= 0),
        defesa_especial INTEGER NOT NULL CHECK (defesa_especial >= 0),
        velocidade INTEGER NOT NULL CHECK (velocidade >= 0),
        id_natureza INTEGER,
        PRIMARY KEY (id_pokedex_pokemon, id_instancia_pokemon),
        FOREIGN KEY (id_treinador) REFERENCES Treinador(id_treinador)
    );
    """,
    """
    CREATE TABLE Item (
        id_item INTEGER,
        id_instancia INTEGER,
        id_treinador INTEGER,
        nome VARCHAR(50) NOT NULL,
        descricao VARCHAR(100),
        PRIMARY KEY (id_item, id_instancia),
        FOREIGN KEY (id_treinador) REFERENCES Treinador(id_treinador)
    );
    """,
    """
    CREATE TABLE Item_Evolucao (
        id_item INTEGER,
        id_instancia INTEGER,
        PRIMARY KEY (id_item, id_instancia),
        FOREIGN KEY (id_item, id_instancia) REFERENCES Item(id_item, id_instancia)
    );
    """,
    """
    CREATE TABLE Item_Batalha (
        id_item INTEGER,
        id_instancia INTEGER,
        valor_efeito_batalha INTEGER,
        PRIMARY KEY (id_item, id_instancia),
        FOREIGN KEY (id_item, id_instancia) REFERENCES Item(id_item, id_instancia)
    );
    """,
    """
    CREATE TABLE Item_Pokebola (
        id_item INTEGER,
        id_instancia INTEGER,
        taxa_captura FLOAT,
        PRIMARY KEY (id_item, id_instancia),
        FOREIGN KEY (id_item, id_instancia) REFERENCES Item(id_item, id_instancia)
    );
    """,
    """
    CREATE TABLE Item_Medicinais (
        id_item INTEGER,
        id_instancia INTEGER,
        valor_efeito_medicinal INTEGER,
        PRIMARY KEY (id_item, id_instancia),
        FOREIGN KEY (id_item, id_instancia) REFERENCES Item(id_item, id_instancia)
    );
    """,
    """
    CREATE TABLE Pokedex (
        id_pokedex SERIAL PRIMARY KEY,
        nome VARCHAR(30),
        descricao VARCHAR(200)
    );
    """,
    """
    CREATE TABLE Equipe (
        id_equipe SERIAL PRIMARY KEY,
        id_treinador INTEGER,
        FOREIGN KEY (id_treinador) REFERENCES Treinador(id_treinador)
    );
    """,
    """
    CREATE TABLE Liga (
        id_liga INTEGER PRIMARY KEY,
        nome VARCHAR(50) NOT NULL
    );
    """,
    """
    CREATE TABLE Pode_Aprender (
        id_pokedex_pokemon INTEGER,
        id_instancia_pokemon INTEGER,
        id_golpe INTEGER,
        Primary KEY (id_pokedex_pokemon, id_instancia_pokemon, id_golpe),
        FOREIGN KEY (id_pokedex_pokemon, id_instancia_pokemon) REFERENCES Pokemon(id_pokedex_pokemon, id_instancia_pokemon)
    );
    """,
    """
    CREATE TABLE Batalha (
        id_treinador_1 INTEGER,
        id_treinador_2 INTEGER,
        vencedor INTEGER,
        PRIMARY KEY (id_treinador_1, id_treinador_2),
        FOREIGN KEY (id_treinador_1) REFERENCES Treinador(id_treinador),
        FOREIGN KEY (id_treinador_2) REFERENCES Treinador(id_treinador),
        FOREIGN KEY (vencedor) REFERENCES Treinador(id_treinador)
    );
    """,
    """
    CREATE TABLE Pokemon_eh_de_tal (
        id_pokedex_pokemon INTEGER,
        id_tipo INTEGER,
        PRIMARY KEY (id_pokedex_pokemon, id_tipo),
        FOREIGN KEY (id_pokedex_pokemon) REFERENCES Pokedex(id_pokedex),
        FOREIGN KEY (id_tipo) REFERENCES Tipo(id_tipo)
    );
    """,
    """
    CREATE TABLE Item_eh_de_tal (
        id_item INTEGER,
        id_instancia INTEGER,
        id_tipo INTEGER,
        PRIMARY KEY (id_item, id_instancia, id_tipo),
        FOREIGN KEY (id_item, id_instancia) REFERENCES Item(id_item, id_instancia),
        FOREIGN KEY (id_tipo) REFERENCES Tipo(id_tipo)
    );
    """,
    """
    CREATE TABLE Eh_formada (
        id_liga INTEGER,
        id_treinador INTEGER,
        PRIMARY KEY (id_liga, id_treinador),
        FOREIGN KEY (id_liga) REFERENCES Liga(id_liga),
        FOREIGN KEY (id_treinador) REFERENCES Treinador(id_treinador)
    );
    """,
    """
    CREATE TABLE Eh_composta (
        id_equipe INTEGER,
        id_pokedex_pokemon INTEGER,
        id_instancia_pokemon INTEGER,
        PRIMARY KEY (id_equipe, id_pokedex_pokemon, id_instancia_pokemon),
        FOREIGN KEY (id_equipe) REFERENCES Equipe(id_equipe),
        FOREIGN key (id_pokedex_pokemon, id_instancia_pokemon) REFERENCES Pokemon(id_pokedex_pokemon, id_instancia_pokemon)
    );
    """,
    """
    CREATE TABLE Possui (
        id_treinador INTEGER,
        id_insignia INTEGER,
        PRIMARY KEY (id_treinador, id_insignia),
        FOREIGN KEY (id_treinador) REFERENCES Treinador(id_treinador),
        FOREIGN KEY (id_insignia) REFERENCES Insignia(id_insignia)
    );
    """,
    """
    CREATE TABLE Sabe (
        id_pokedex_pokemon INTEGER,
        id_instancia_pokemon INTEGER,
        id_golpe INTEGER,
        PRIMARY KEY (id_pokedex_pokemon, id_instancia_pokemon, id_golpe),
        FOREIGN KEY (id_pokedex_pokemon, id_instancia_pokemon) REFERENCES Pokemon(id_pokedex_pokemon, id_instancia_pokemon),
        FOREIGN KEY (id_golpe) REFERENCES Golpe(id_golpe)
    );
    """
]

# Executando os comandos
for command in sql_commands:
    cursor.execute(command)

print("Tabelas do banco de dados criadas com sucesso!")

Tabelas do banco de dados criadas com sucesso!


In [6]:
# Criação de Index no banco de dados
sql_commands = [
"""
-- Índice para consulta 1 - Contagem de vitórias por treinador
CREATE INDEX idx_batalha_vencedor ON Batalha(vencedor);
""",
"""
CREATE INDEX idx_treinador_nome ON Treinador(nome);
""",
"""
-- Índice para consulta 2 - Treinadores com Pokémon de ataque alto
CREATE INDEX idx_pokemon_ataque ON Pokemon(ataque);
""",
"""
CREATE INDEX idx_pokemon_ataque_especial ON Pokemon(ataque_especial);
""",
"""
CREATE INDEX idx_equipe_treinador ON Equipe(id_treinador);
""",
"""
CREATE INDEX idx_eh_composta_equipe ON Eh_composta(id_equipe);
""",
"""
-- Índice para consulta 3 - Combinações de tipos de Pokémon
CREATE INDEX idx_pokemon_eh_de_tal_pokemon ON Pokemon_eh_de_tal(id_pokedex_pokemon);
""",
"""
CREATE INDEX idx_tipo_nome ON Tipo(nome);
""",
"""
-- Índice para consulta 4 - Golpes por tipo
CREATE INDEX idx_golpe_tipo ON Golpe(id_tipo);
""",
"""
-- Índice para consulta 5 - Golpes conhecidos por Pokémon
CREATE INDEX idx_sabe_golpe ON Sabe(id_golpe);
""",
"""
CREATE INDEX idx_pokemon_nome ON Pokemon(nome);
""",
"""
-- Índice para consulta 6 - Pokémon que evoluem com pedras
CREATE INDEX idx_item_nome ON Item(nome);
""",
"""
CREATE INDEX idx_item_evolucao_item ON Item_Evolucao(id_item);
""",
"""
CREATE INDEX idx_pokemon_evolui_com_pedra ON Pokemon(evolui_com_pedra);
""",
"""
-- Índice para consulta 7 - Pokémon com múltiplos tipos
CREATE INDEX idx_pokemon_eh_de_tal_tipo ON Pokemon_eh_de_tal(id_tipo);
""",
"""
-- Índice para consulta 8 - Insígnias de treinadores
CREATE INDEX idx_possui_insignia ON Possui(id_insignia);
""",
"""
CREATE INDEX idx_insignia_nome ON Insignia(nome);
""",
"""
-- Índice para consulta 9 - Golpes com precisão menor que 100
CREATE INDEX idx_golpe_precisao ON Golpe(precisao);
""",
"""
-- Índice para consulta 10 - Treinadores com Pokémon de nível alto
CREATE INDEX idx_pokemon_nivel ON Pokemon(nivel);
""",
"""
-- Índice para consulta 11 - Total de Pokémon capturados por treinador
CREATE INDEX idx_pokemon_treinador ON Pokemon(id_treinador);
"""
]

# Executando os comandos
for command in sql_commands:
    cursor.execute(command)

print("Indexações criadas com sucesso!")

Indexações criadas com sucesso!


In [7]:
# Criação de Views no banco de dados
sql_commands = [
"""
-- View 1 -- Mostra pokemon e seu treinador e null se pokemon não tiver treinador
CREATE OR REPLACE VIEW treinador_pokemon AS
SELECT 
	t.id_treinador, 
    p.id_pokedex_pokemon, 
    p.id_instancia_pokemon
FROM 
	Pokemon P 
LEFT JOIN 
	Treinador T on t.id_treinador = p.id_treinador;
""",
"""
-- View 2 -- Mostra treinador, pokemon e seus atributos e golpes
CREATE OR REPLACE VIEW interface_de_batalha AS
SELECT 
    t.nome AS nome_treinador,  
    p.nome AS nome_pokemon,    
    p.hp, 
    p.nivel, 
    STRING_AGG(s.id_golpe::TEXT, ', ') AS golpes
FROM 
    Treinador t 
JOIN 
    Pokemon p 
ON 
    p.id_treinador = t.id_treinador
JOIN 
    Sabe s 
ON 
    p.id_pokedex_pokemon = s.id_pokedex_pokemon 
    AND p.id_instancia_pokemon = s.id_instancia_pokemon
GROUP BY 
    p.id_pokedex_pokemon, 
    p.id_instancia_pokemon, 
    t.nome, 
    p.nome, 
    p.hp, 
    p.nivel;
""",
"""
-- View 3 -- Mostra pokemon e seus tipos
CREATE OR REPLACE VIEW pokemon_com_tipos AS
SELECT 
    p.id_pokedex_pokemon,
    p.nome,
    ARRAY_AGG(DISTINCT t.id_tipo) AS id_tipo,
    STRING_AGG(DISTINCT t.nome, ', ') AS tipo
FROM 
    Pokemon p
LEFT JOIN Pokemon_eh_de_tal pet ON pet.id_pokedex_pokemon = p.id_pokedex_pokemon
LEFT JOIN Tipo t ON t.id_tipo = pet.id_tipo
GROUP BY 
    p.id_pokedex_pokemon, 
    p.nome;
"""
]

# Executando os comandos
for command in sql_commands:
    cursor.execute(command)

print("Views criadas com sucesso!")

Views criadas com sucesso!


In [8]:
# Criação de Procedures e Functions no banco de dados
sql_commands = [
"""
-- Funcao 1 - Captura pokemon, checa se ele já tem treinador e se o treinador tentando capturá-lo tem pokebolas
CREATE OR REPLACE FUNCTION captura_pokemon(
    id_pokedex_pokemon1 INTEGER,
    id_instancia_pokemon1 INTEGER,
    id_treinador1 INTEGER,
    id_item1 INTEGER,
    id_instancia_pokebola1 INTEGER
) RETURNS BOOLEAN AS $$
DECLARE
    possui_pokebola BOOLEAN;
    pokemon_disponivel BOOLEAN;
BEGIN
    SELECT EXISTS (
        SELECT 1
        FROM item_pokebola IPKB
        JOIN item I ON I.id_item = IPKB.id_item AND I.id_instancia = IPKB.id.instancia
        WHERE I.id_treinador = id_treinador1 AND I.id_item = id_item1 AND I.id_instancia = id_instancia_pokebola1
    ) INTO possui_pokebola;
    SELECT EXISTS (
        SELECT 1
        FROM Pokemon
        WHERE id_pokedex_pokemon = id_pokedex_pokemon1
          AND id_instancia_pokemon = id_instancia_pokemon1
          AND id_treinador IS NULL
    ) INTO pokemon_disponivel;

    IF possui_pokebola AND pokemon_disponivel THEN
        UPDATE Pokemon
        SET id_treinador = id_treinador1
        WHERE id_pokedex_pokemon = id_pokedex_pokemon1
          AND id_instancia_pokemon = id_instancia_pokemon1;

        DELETE FROM Item 
        WHERE I.id_item = id.item1 AND I.id_instancia = id_instancia_pokebola1;

        RETURN TRUE; 
    ELSE
        RETURN FALSE; 
    END IF;
END;
$$ LANGUAGE plpgsql;
""",
"""
-----------------------------------------------------------
-- Funcao 2  - Registra batalha e verifica se os treinadores existem
CREATE OR REPLACE FUNCTION registrar_batalha(
    id_treinador_1 INTEGER,
    id_treinador_2 INTEGER,
    vencedor INTEGER
) RETURNS VOID AS $$
BEGIN
    IF EXISTS (
        SELECT 1 
        FROM Treinador 
        WHERE id_treinador = id_treinador_1
    ) THEN
        IF EXISTS (
            SELECT 1 
            FROM Treinador 
            WHERE id_treinador = id_treinador_2
        ) THEN
            IF vencedor = id_treinador_1 OR vencedor = id_treinador_2 
THEN
                INSERT INTO Batalha (id_treinador_1, id_treinador_2, vencedor)
                VALUES (id_treinador_1, id_treinador_2, vencedor);
            ELSE
                RAISE EXCEPTION 'Erro: O vencedor deve ser um dos treinadores participantes da batalha.';
            END IF;
        ELSE
            RAISE EXCEPTION 'Erro: O Treinador 2 não existe.';
        END IF;
    ELSE
        RAISE EXCEPTION 'Erro: O Treinador 1 não existe.';
    END IF;
END;
$$ LANGUAGE plpgsql;
""",
"""
-----------------------------------------------------------
-- Funcao 3 - Se tem 4 golpes não pode colocar mais
CREATE OR REPLACE FUNCTION adicionar_golpe(
    id_pokedex_pokemon1 INTEGER,
    id_instancia_pokemon1 INTEGER,
    id_golpe1 INTEGER
) RETURNS VOID AS $$
BEGIN
    IF EXISTS (
        SELECT 1
        FROM Pode_Aprender
        WHERE id_pokedex_pokemon = id_pokedex_pokemon1
          AND id_golpe = id_golpe1
    ) 
    AND NOT EXISTS (
        SELECT 1
        FROM Sabe
        WHERE id_pokedex_pokemon = id_pokedex_pokemon1
          AND id_instancia_pokemon = id_instancia_pokemon1
          AND id_golpe = id_golpe1
    ) 
    AND (
        SELECT COUNT(*)
        FROM Sabe
        WHERE id_pokedex_pokemon = id_pokedex_pokemon1
          AND id_instancia_pokemon = id_instancia_pokemon1
    ) < 4 THEN
        INSERT INTO Sabe (id_pokedex_pokemon, id_instancia_pokemon, id_golpe)
        VALUES (id_pokedex_pokemon1, id_instancia_pokemon1, id_golpe1);
    ELSE
        RAISE NOTICE 'Não foi possível adicionar o golpe.';
    END IF;
END;
$$ LANGUAGE plpgsql;
""",
"""
-----------------------------------------------------------
-- Função 4 - Para subir de nível e ajustar os atributos
CREATE OR REPLACE FUNCTION subir_de_nivel(
    p_id_pokedex_pokemon INTEGER,
    p_id_instancia_pokemon INTEGER
)
RETURNS VARCHAR AS $$
DECLARE
    nivel_atual INTEGER;
    nivel_maximo CONSTANT INTEGER := 100;
BEGIN
    SELECT nivel INTO nivel_atual
    FROM Pokemon
    WHERE id_pokedex_pokemon = p_id_pokedex_pokemon 
      AND id_instancia_pokemon = p_id_instancia_pokemon;

    IF nivel_atual >= nivel_maximo THEN
        RETURN 'O Pokémon não pode mais subir de nível, pois está no nível máximo.';
    END IF;

    UPDATE Pokemon
    SET 
        nivel = nivel + 1,
        hp = hp + 2,
        ataque = ataque + 2,
        defesa = defesa + 2,
        ataque_especial = ataque_especial + 2,
        defesa_especial = defesa_especial + 2,
        velocidade = velocidade + 2
    WHERE id_pokedex_pokemon = p_id_pokedex_pokemon 
      AND id_instancia_pokemon = p_id_instancia_pokemon;

    RETURN FORMAT('O Pokémon subiu para o nível %s.', nivel_atual + 1);
END;
$$ LANGUAGE plpgsql;
""",
"""
------------------------------------------------------------------
-- Funcao 5 - Transfere pokemon e verifica treinadores
CREATE OR REPLACE FUNCTION transferir_pokemon(
    id_pokedex_pokemon1 INTEGER,
    id_instancia_pokemon1 INTEGER,
    id_treinador_origem INTEGER,
    id_treinador_destino INTEGER
) RETURNS TEXT AS $$
DECLARE
    treinador_atual INTEGER;
    equipe_origem INTEGER;
    equipe_destino INTEGER;
BEGIN
    SELECT id_treinador
    INTO treinador_atual
    FROM Pokemon
    WHERE id_pokedex_pokemon = id_pokedex_pokemon1
      AND id_instancia_pokemon = id_instancia_pokemon1;

    IF treinador_atual IS NULL THEN
        RETURN 'Erro: Este Pokémon não pertence a nenhum treinador.';
    ELSIF treinador_atual != id_treinador_origem THEN
        RETURN 'Erro: O Pokémon não pertence ao treinador de origem.';
    END IF;

    IF NOT EXISTS (
        SELECT 1
        FROM Treinador
        WHERE id_treinador = id_treinador_destino
    ) THEN
        RETURN 'Erro: Treinador de destino não encontrado.';
    END IF;

    SELECT id_equipe INTO equipe_origem
    FROM Equipe
    WHERE id_treinador = id_treinador_origem;

    IF equipe_origem IS NOT NULL THEN
        DELETE FROM Eh_composta
        WHERE id_equipe = equipe_origem
          AND id_pokedex_pokemon = id_pokedex_pokemon1
          AND id_instancia_pokemon = id_instancia_pokemon1;
    END IF;

    SELECT id_equipe INTO equipe_destino
    FROM Equipe
    WHERE id_treinador = id_treinador_destino;

    IF equipe_destino IS NOT NULL THEN
        IF (
            SELECT COUNT(*)
            FROM Eh_composta
            WHERE id_equipe = equipe_destino
        ) >= 6 THEN
            RETURN 'Erro: O treinador de destino já possui o limite de 6 Pokémon na equipe.';
        END IF;

        INSERT INTO Eh_composta (id_equipe, id_pokedex_pokemon, id_instancia_pokemon)
        VALUES (equipe_destino, id_pokedex_pokemon1, id_instancia_pokemon1);
    END IF;

    UPDATE Pokemon
    SET id_treinador = id_treinador_destino
    WHERE id_pokedex_pokemon = id_pokedex_pokemon1
      AND id_instancia_pokemon = id_instancia_pokemon1;

    RETURN FORMAT(
        'Transferência concluída! Pokémon %s (instância %s) agora pertence ao Treinador %s.',
        id_pokedex_pokemon1, id_instancia_pokemon1, id_treinador_destino
    );
END;
$$ LANGUAGE plpgsql;
""",
"""
------------------------------------------------------------------
-- Funcao 6 - Adiciona pokemon a equipe
CREATE OR REPLACE FUNCTION adicionar_pokemon_equipe(
    id_equipe1 INTEGER,
    id_pokedex_pokemon1 INTEGER,
    id_instancia_pokemon1 INTEGER
)
RETURNS VARCHAR AS $$
DECLARE
    mensagem VARCHAR;
BEGIN
    IF NOT EXISTS(
		SELECT 1 
		FROM Equipe
		WHERE id_equipe = id_equipe1) 
    AND NOT EXISTS(
		SELECT 1
		FROM Pokemon 
		WHERE id_pokedex_pokemon = id_pokedex_pokemon1 AND id_instancia_pokemon = id_instancia_pokemon1)
    THEN 
    	RETURN 'Erro: Dados inconsistentes!';
    END IF;
    IF EXISTS (
      SELECT 1
      FROM Eh_composta
        WHERE id_equipe = id_equipe1
          AND id_pokedex_pokemon = id_pokedex_pokemon1
          AND id_instancia_pokemon = id_instancia_pokemon1
    ) THEN
        RETURN 'Erro: Pokémon já está na equipe.';
    END IF;

    IF (
        SELECT COUNT(*)
        FROM Eh_composta
        WHERE id_equipe = id_equipe
    ) >= 6 THEN
        RETURN 'Erro: A equipe já possui o limite de 6 Pokémon.';
    END IF;

    INSERT INTO Eh_composta (id_equipe, id_pokedex_pokemon, id_instancia_pokemon)
    VALUES (id_equipe, id_pokedex_pokemon, id_instancia_pokemon);

    mensagem := 'Pokémon (ID: ' || id_pokedex_pokemon || ', Instância: ' || id_instancia_pokemon ||
                 ') foi adicionado à equipe (ID: ' || id_equipe || ').';

    RETURN mensagem;
END;
$$ LANGUAGE plpgsql;
"""
]

# Executando os comandos
for command in sql_commands:
    cursor.execute(command)

print("Procedures/Functions criadas com sucesso!")

Procedures/Functions criadas com sucesso!


In [9]:
# Criação de Triggers no banco de dados
sql_commands = [
"""
CREATE OR REPLACE FUNCTION verifica_equipe_apos_troca()
RETURNS TRIGGER AS $$
BEGIN
    -- Verifica se o Pokémon antigo faz parte de uma equipe
    IF EXISTS (
        SELECT 1
        FROM Eh_composta ec
        WHERE ec.id_pokedex_pokemon = OLD.id_pokedex_pokemon
          AND ec.id_instancia_pokemon = OLD.id_instancia_pokemon
    ) THEN
        -- Remove o Pokémon da equipe
        DELETE FROM Eh_composta
        WHERE id_pokedex_pokemon = OLD.id_pokedex_pokemon
          AND id_instancia_pokemon = OLD.id_instancia_pokemon;
    END IF;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

-- Trigger para remover Pokémon de uma equipe após a troca
CREATE TRIGGER trg_remove_pokemon_da_equipe
AFTER UPDATE OF id_treinador ON Pokemon
FOR EACH ROW
WHEN (OLD.id_treinador IS DISTINCT FROM NEW.id_treinador)
EXECUTE FUNCTION verifica_equipe_apos_troca();
""",
"""
------------------------------------------------------------------
CREATE OR REPLACE FUNCTION verifica_participacao_liga()
RETURNS TRIGGER AS $$
BEGIN
    -- Verifica se o treinador adversário pertence à Liga
    IF EXISTS (
        SELECT 1
        FROM Eh_formada ef
        WHERE ef.id_treinador = NEW.id_treinador_2
    ) THEN
        -- Verifica se o treinador tem as 8 insígnias necessárias
        IF (
            SELECT COUNT(*)
            FROM Possui p
            WHERE p.id_treinador = NEW.id_treinador_1
        ) < 8 THEN
            RAISE EXCEPTION 'Treinador precisa ter as 8 insígnias para participar da liga.';
        END IF;
    END IF;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER trg_verifica_participacao_liga
BEFORE INSERT ON Batalha
FOR EACH ROW
EXECUTE FUNCTION verifica_participacao_liga();
""",
"""
------------------------------------------------------------------
CREATE OR REPLACE FUNCTION verifica_limite_equipe()
RETURNS TRIGGER AS $$
BEGIN
    -- Verifica se o treinador já possui 6 Pokémon na equipe associada
    IF (
        SELECT COUNT(*)
        FROM Eh_composta ec
        JOIN Equipe e ON ec.id_equipe = e.id_equipe
        WHERE e.id_treinador = (
            SELECT id_treinador 
            FROM Equipe 
            WHERE id_equipe = NEW.id_equipe
        )
    ) >= 6 THEN
        RAISE EXCEPTION 'Treinador já possui o limite de 6 Pokémon na equipe.';
    END IF;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

-- Trigger para verificar limite de Pokémon na equipe
CREATE OR REPLACE TRIGGER trg_verifica_limite_equipe
BEFORE INSERT ON Eh_composta
FOR EACH ROW
EXECUTE FUNCTION verifica_limite_equipe();
"""
]

# Executando os comandos
for command in sql_commands:
    cursor.execute(command)
    
print("Triggers criadas com sucesso!")

Triggers criadas com sucesso!


In [10]:
# Inserções no banco de dados
sql_commands = [
"""
INSERT INTO Tipo (id_tipo, nome) VALUES
(1, 'Normal'),
(2, 'Fighting'),
(3, 'Flying'),
(4, 'Bug'),
(5, 'Ground'),
(6, 'Rock'),
(7, 'Ghost'),
(8, 'Poison'),
(10, 'Fire'),
(11, 'Water'),
(12, 'Grass'),
(13, 'Electric'),
(14, 'Psychic'),
(15, 'Ice'),
(16, 'Dragon');

INSERT INTO Pokedex (id_pokedex, nome, descricao) VALUES
(1, 'Bulbasaur', 'Uma semente estranha foi plantada em suas costas ao nascer. A planta cresce junto com o Pokémon.'),
(2, 'Ivysaur', 'Quando o bulbo em suas costas cresce, parece perder a capacidade de andar sobre as patas traseiras.'),
(3, 'Venusaur', 'A planta floresce quando absorve energia solar. Ele se move para buscar a luz do sol.'),
(4, 'Charmander', 'Prefere lugares quentes. Quando chove, é dito que vapor sai da ponta de sua cauda.'),
(5, 'Charmeleon', 'Quando balança sua cauda em chamas, a temperatura aumenta a níveis insuportáveis.'),
(6, 'Charizard', 'Cospe fogo quente o suficiente para derreter rochas. Pode causar incêndios florestais acidentalmente.'),
(7, 'Squirtle', 'Após nascer, suas costas incham e endurecem formando uma concha. Ele esguicha água pela boca.'),
(8, 'Wartortle', 'Geralmente se esconde na água para caçar presas desprevenidas. Move suas orelhas para manter o equilíbrio.'),
(9, 'Blastoise', 'Um Pokémon brutal com jatos de água de alta pressão em sua concha, usados para tackles rápidos.'),
(10, 'Caterpie', 'Suas patas possuem ventosas que permitem escalar superfícies inclinadas e paredes sem esforço.'),
(11, 'Metapod', 'Este Pokémon é vulnerável enquanto sua casca está mole, expondo seu corpo frágil.'),
(12, 'Butterfree', 'Em batalha, bate suas asas rapidamente para liberar poeira tóxica no ar.'),
(13, 'Weedle', 'Frequentemente encontrado em florestas, comendo folhas. Possui um ferrão venenoso na cabeça.'),
(14, 'Kakuna', 'Quase incapaz de se mover, endurece sua casca para se proteger dos predadores.'),
(15, 'Beedrill', 'Voa rapidamente e ataca com seus ferrões venenosos nas patas dianteiras e cauda.'),
(16, 'Pidgey', 'Comum em florestas e bosques. Bate as asas no solo para levantar areia cegante.'),
(17, 'Pidgeotto', 'Muito protetor de seu território, ataca ferozmente intrusos com bicadas.'),
(18, 'Pidgeot', 'Caça sobrevoando a água em alta velocidade para pegar presas como Magikarp.'),
(19, 'Rattata', 'Morde qualquer coisa quando ataca. Pequeno e muito rápido, é visto em muitos lugares.'),
(20, 'Raticate', 'Usa seus bigodes para manter o equilíbrio. Parece desacelerar quando eles são cortados.'),
(21, 'Spearow', 'Come insetos em áreas gramadas. Suas asas curtas precisam bater rapidamente para mantê-lo no ar.'),
(22, 'Fearow', 'Com suas asas enormes, pode se manter no ar por longos períodos sem precisar pousar.'),
(23, 'Ekans', 'Move-se silenciosamente. Come ovos de pássaros como Pidgey e Spearow inteiros.'),
(24, 'Arbok', 'Os padrões ameaçadores em sua barriga variam de região para região.'),
(25, 'Pikachu', 'Quando vários Pikachu se reúnem, sua eletricidade pode causar tempestades.'),
(26, 'Raichu', 'Sua cauda longa serve como aterramento para protegê-lo de sua própria eletricidade.'),
(27, 'Sandshrew', 'Escava profundamente em áreas áridas e longe da água. Só emerge para caçar comida.'),
(28, 'Sandslash', 'Enrola-se em uma bola espinhosa quando ameaçado. Pode rolar nessa forma para atacar.'),
(29, 'Nidoran♀', 'Apesar de pequeno, seus espinhos venenosos o tornam perigoso. A fêmea tem chifres menores.'),
(30, 'Nidorina', 'O chifre da fêmea se desenvolve lentamente. Prefere ataques físicos como arranhões e mordidas.'),
(31, 'Nidoqueen', 'Suas escamas duras proporcionam grande proteção. Usa seu corpo robusto para golpes poderosos.'),
(32, 'Nidoran♂', 'Endurece suas orelhas para detectar perigo. Quanto maiores os chifres, mais poderoso o veneno.'),
(33, 'Nidorino', 'Um Pokémon agressivo e rápido para atacar. Seu chifre excreta um veneno poderoso.'),
(34, 'Nidoking', 'Usa sua cauda poderosa para lutar, esmagar e quebrar os ossos da presa.'),
(35, 'Clefairy', 'Seu charme mágico atrai muitos admiradores. É raro e encontrado apenas em certas áreas.'),
(36, 'Clefable', 'Um Pokémon fada tímido que raramente é visto. Foge ao primeiro sinal de pessoas.'),
(37, 'Vulpix', 'Ao nascer, tem apenas uma cauda. A cauda se divide conforme envelhece.'),
(38, 'Ninetales', 'Muito inteligente e vingativo. Pegar uma de suas caudas pode resultar em uma maldição de 1000 anos.'),
(39, 'Jigglypuff', 'Quando seus olhos brilham, canta uma melodia relaxante que adormece os oponentes.'),
(40, 'Wigglytuff', 'Seu corpo é macio e flexível. Quando irritado, infla-se a um tamanho enorme.'),
(41, 'Zubat', 'Forma colônias em lugares escuros. Usa ondas ultrassônicas para localizar presas.'),
(42, 'Golbat', 'Depois de atacar, não para de sugar a energia da vítima, mesmo se ficar pesado demais para voar.'),
(43, 'Oddish', 'Durante o dia, mantém o rosto enterrado no solo. À noite, anda espalhando suas sementes.'),
(44, 'Gloom', 'O fluido que escorre de sua boca não é baba, mas néctar usado para atrair presas.'),
(45, 'Vileplume', 'Quanto maiores suas pétalas, mais pólen tóxico contêm. Sua cabeça grande é pesada e difícil de sustentar.'),
(46, 'Paras', 'Enterra-se para sugar raízes de árvores. Os cogumelos crescem em suas costas, extraindo nutrientes do Pokémon.'),
(47, 'Parasect', 'Um par hospedeiro-parasita onde o cogumelo tomou conta do corpo do inseto. Prefere locais úmidos.'),
(48, 'Venonat', 'Vive nas sombras de árvores altas, onde se alimenta de insetos.'),
(49, 'Venomoth', 'Ativo à noite, come insetos atraídos pela luz. Suas asas são cobertas de escamas tóxicas.'),
(50, 'Diglett', 'Vive a cerca de um metro de profundidade no solo, onde se alimenta de raízes de plantas.'),
(51, 'Dugtrio', 'Uma equipe de três Digletts cavando em uníssono pode causar terremotos.'),
(52, 'Meowth', 'Ama coisas redondas e brilhantes. As noites passam caçando objetos brilhantes para colecionar.'),
(53, 'Persian', 'Muito orgulhoso de seu pelo fino e brilhante. Ele anda elegante como se estivesse desfilando.'),
(54, 'Psyduck', 'É frequentemente incomodado por dores de cabeça. Quando as dores aumentam, ele libera poderes psíquicos.'),
(55, 'Golduck', 'Conhecido por suas habilidades de natação, ele nada rapidamente até em águas turbulentas.'),
(56, 'Mankey', 'Muito irritadiço. Quando começa a se enfurecer, não consegue parar.'),
(57, 'Primeape', 'Entra em fúria tão intensa que não consegue distinguir amigos de inimigos.'),
(58, 'Growlithe', 'Extremamente leal. Protegerá seu treinador até o fim, mesmo enfrentando oponentes maiores.'),
(59, 'Arcanine', 'Um Pokémon lendário famoso por sua velocidade e nobreza. Diz-se que cruza continentes em um dia.'),
(60, 'Poliwag', 'Sua pele é tão fina que se podem ver seus órgãos. O espiral no seu estômago é, na verdade, seus intestinos.'),
(61, 'Poliwhirl', 'Seu corpo escorregadio o protege de ser agarrado durante batalhas.'),
(62, 'Poliwrath', 'Suas poderosas pernas permitem nadar grandes distâncias com facilidade.'),
(63, 'Abra', 'Passa 18 horas do dia dormindo. Enquanto dorme, usa seus poderes psíquicos para se proteger.'),
(64, 'Kadabra', 'Diz-se que uma criança com poderes psíquicos despertou um dia e se transformou em Kadabra.'),
(65, 'Alakazam', 'Suas habilidades psíquicas são incríveis. Ele pode se lembrar de cada detalhe de sua vida inteira.'),
(66, 'Machop', 'Ama treinar seus músculos e pode levantar 1000 pessoas ao mesmo tempo.'),
(67, 'Machoke', 'Seus músculos são tão poderosos que precisa de um cinto especial para conter sua força.'),
(68, 'Machamp', 'Com quatro braços poderosos, pode realizar múltiplos socos ao mesmo tempo.'),
(69, 'Bellsprout', 'Prefere ambientes úmidos e quentes. Se alimenta de pequenos insetos usando sua boca.'),
(70, 'Weepinbell', 'Libera um pó tóxico de sua boca. Se se sente ameaçado, libera a substância instantaneamente.'),
(71, 'Victreebel', 'Atrai presas com um aroma doce e as dissolve lentamente com ácido.'),
(72, 'Tentacool', 'Flutua perto da superfície do mar. Se sua pele secar, ele retornará ao oceano.'),
(73, 'Tentacruel', 'Com seus tentáculos, pode capturar e imobilizar a presa facilmente.'),
(74, 'Geodude', 'Costuma ser confundido com rochas. É encontrado frequentemente em montanhas e cavernas.'),
(75, 'Graveler', 'Desce colinas rolando e quebrando tudo em seu caminho.'),
(76, 'Golem', 'Seu corpo é tão duro quanto uma rocha. Pode sobreviver até a uma erupção vulcânica.'),
(77, 'Ponyta', 'Suas pernas ficam fortes ao correr por planícies. É capaz de saltar sobre edifícios altos.'),
(78, 'Rapidash', 'Pode correr a uma velocidade incrível, suas chamas brilham quando atinge a máxima velocidade.'),
(79, 'Slowpoke', 'Muito lento e distraído. Às vezes, esquece o que estava fazendo.'),
(80, 'Slowbro', 'Quando um Shellder morde sua cauda, ele se torna mais forte e focado.'),
(81, 'Magnemite', 'Gera campos magnéticos poderosos. Fica preso a objetos metálicos.'),
(82, 'Magneton', 'Um grupo de três Magnemite. Sua presença pode causar interferências elétricas.'),
(83, 'Farfetch’d', 'Usa o talo de alho-poró como arma. É raro e muito difícil de encontrar na natureza.'),
(84, 'Doduo', 'Possui duas cabeças e pode correr rapidamente. Ambas as cabeças se alternam para vigiar.'),
(85, 'Dodrio', 'Com três cabeças, pode realizar ataques coordenados. Suas pernas são incrivelmente rápidas.'),
(86, 'Seel', 'Adora nadar em águas frias. Seu chifre afiado pode quebrar gelo com facilidade.'),
(87, 'Dewgong', 'Nada graciosamente em águas geladas. Sua pele é isolante e protege do frio intenso.'),
(88, 'Grimer', 'Nascido de lodo tóxico. Deixa um rastro contaminado por onde passa.'),
(89, 'Muk', 'É feito de uma substância tóxica que pode poluir rios e lagos.'),
(90, 'Shellder', 'Sua concha é extremamente dura. Protege seu corpo macio dos predadores.'),
(91, 'Cloyster', 'Pode se fechar completamente, tornando-se quase impenetrável.'),
(92, 'Gastly', 'Feito de um gás venenoso. Pode atravessar paredes e superfícies sólidas.'),
(93, 'Haunter', 'Flutua no ar e pode causar paralisia apenas tocando a vítima.'),
(94, 'Gengar', 'Adora pregar peças. Às vezes, se esconde nas sombras para assustar pessoas.'),
(95, 'Onix', 'Seus segmentos de pedra formam um corpo longo e resistente.'),
(96, 'Drowzee', 'Pode induzir o sono e se alimenta dos sonhos de seus alvos.'),
(97, 'Hypno', 'Usa seu pêndulo para hipnotizar suas presas e induzir o sono.'),
(98, 'Krabby', 'Vive em praias e margens de rios. Usa suas garras para se proteger.'),
(99, 'Kingler', 'Sua garra gigante é tão pesada que às vezes dificulta seu equilíbrio.'),
(100, 'Voltorb', 'Frequentemente confundido com uma Pokébola. Explode ao ser tocado.'),
(101, 'Electrode', 'Acumula energia elétrica e pode explodir de repente.'),
(102, 'Exeggcute', 'Consiste em seis ovos que se comunicam telepaticamente.'),
(103, 'Exeggutor', 'Suas cabeças falam independentemente, mas cooperam em grupo.'),
(104, 'Cubone', 'Usa o osso de sua mãe falecida como arma. É um Pokémon muito solitário.'),
(105, 'Marowak', 'Superou a dor da perda. Usa o osso para executar ataques poderosos.'),
(106, 'Hitmonlee', 'Especialista em chutes. Suas pernas podem se estender para ataques de longo alcance.'),
(107, 'Hitmonchan', 'Especialista em socos. Suas técnicas de boxe são incríveis.'),
(108, 'Lickitung', 'Sua língua pegajosa pode se estender até o dobro do tamanho de seu corpo.'),
(109, 'Koffing', 'Expele gases tóxicos. É encontrado em locais poluídos.'),
(110, 'Weezing', 'Formado por dois Koffing fundidos. Seus gases são ainda mais tóxicos.'),
(111, 'Rhyhorn', 'Seu corpo blindado é capaz de resistir a impactos poderosos.'),
(112, 'Rhydon', 'Seus chifres perfuram até rochas. Sua pele é resistente como armadura.'),
(113, 'Chansey', 'Um Pokémon gentil que compartilha ovos nutritivos para ajudar os feridos.'),
(114, 'Tangela', 'Coberto por vinhas azuis. Ninguém jamais viu seu verdadeiro corpo.'),
(115, 'Kangaskhan', 'Cuida do filhote em sua bolsa. Extremamente protetora e feroz.'),
(116, 'Horsea', 'Libera jatos de tinta quando está em perigo para escapar.'),
(117, 'Seadra', 'Navega pelas correntes marinhas. Suas nadadeiras são afiadas como lâminas.'),
(118, 'Goldeen', 'Nada elegantemente, suas barbatanas se movem como um vestido fluido.'),
(119, 'Seaking', 'Durante a estação de acasalamento, nadam juntos em padrões hipnotizantes.'),
(120, 'Staryu', 'Mesmo se perder uma das pontas de seu corpo em forma de estrela, pode regenerá-la.'),
(121, 'Starmie', 'Suas gemas centrais brilham de forma mística. Comunica-se usando luz piscante.'),
(122, 'Mr. Mime', 'Especialista em mímica. Cria barreiras invisíveis imitando gestos.'),
(123, 'Scyther', 'Seus braços afiados como lâminas podem cortar oponentes com facilidade.'),
(124, 'Jynx', 'Dança de forma hipnótica para confundir os oponentes durante a batalha.'),
(125, 'Electabuzz', 'A eletricidade em seu corpo causa interferências em aparelhos eletrônicos.'),
(126, 'Magmar', 'O calor que emana de seu corpo pode derreter pedras e incendiar florestas.'),
(127, 'Pinsir', 'Aperta seus inimigos com suas pinças poderosas até que parem de se mover.'),
(128, 'Tauros', 'Corre por campos enquanto chicoteia o próprio corpo com suas caudas.'),
(129, 'Magikarp', 'Extremamente fraco e inútil, mas pode pular incrivelmente alto.'),
(130, 'Gyarados', 'Após evoluir, torna-se violento e destrutivo, frequentemente atacando cidades.'),
(131, 'Lapras', 'Gentil e inteligente, adora carregar pessoas em suas costas pelos mares.'),
(132, 'Ditto', 'Pode se transformar em qualquer objeto ou Pokémon, copiando sua aparência.'),
(133, 'Eevee', 'Possui uma estrutura genética instável, permitindo evoluções variadas.'),
(134, 'Vaporeon', 'Sua pele se mistura com a água, tornando-se praticamente invisível.'),
(135, 'Jolteon', 'Gera eletricidade estática e pode disparar agulhas de seus pelos.'),
(136, 'Flareon', 'Seu corpo armazena calor intenso, capaz de exalar chamas a 900°C.'),
(137, 'Porygon', 'Um Pokémon artificial criado por humanos através de programação de computador.'),
(138, 'Omanyte', 'Fóssil restaurado. Possui tentáculos usados para nadar.'),
(139, 'Omastar', 'Seus tentáculos são usados para capturar presas e se mover lentamente.'),
(140, 'Kabuto', 'Reviveu de um fóssil. Costuma se esconder na areia em águas rasas.'),
(141, 'Kabutops', 'Seus braços afiados como lâminas são usados para cortar presas.'),
(142, 'Aerodactyl', 'Reviveu de um fóssil. Conhecido por seu voo rápido e mordidas poderosas.'),
(143, 'Snorlax', 'Passa o dia dormindo e comendo. Difícil de acordar quando está em sono profundo.'),
(144, 'Articuno', 'Lendário pássaro de gelo. Diz-se que traz o inverno ao bater as asas.'),
(145, 'Zapdos', 'Lendário pássaro elétrico. Suas asas soltam faíscas ao bater.'),
(146, 'Moltres', 'Lendário pássaro de fogo. Suas asas brilham intensamente em chamas.'),
(147, 'Dratini', 'Raro e místico, vive escondido em profundezas aquáticas.'),
(148, 'Dragonair', 'Seu corpo longo e elegante emite uma aura misteriosa.'),
(149, 'Dragonite', 'Pode voar ao redor do mundo em um dia. Muito inteligente e amigável.'),
(150, 'Mewtwo', 'Criado geneticamente. Seu poder psíquico é inigualável.'),
(151, 'Mew', 'Dizem ser o ancestral de todos os Pokémon. Possui DNA de cada espécie.');

INSERT INTO Natureza (id_natureza, descricao, efeito) VALUES
(1, 'Hardy', 'Neutro'),
(2, 'Lonely', '+Ataque,-Defesa'),
(3, 'Brave', '+Ataque,-Velocidade'),
(4, 'Adamant', '+Ataque,-Ataque Especial'),
(5, 'Naughty', '+Ataque,-Defesa Especial'),
(6, 'Bold', '+Defesa,-Ataque'),
(7, 'Docile', 'Neutro'),
(8, 'Relaxed', '+Defesa,-Velocidade'),
(9, 'Impish', '+Defesa,-Ataque Especial'),
(10, 'Lax', '+Defesa,-Defesa Especial'),
(11, 'Timid', '+Velocidade,-Ataque'),
(12, 'Hasty', '+Velocidade,-Defesa'),
(13, 'Serious', 'Neutro'),
(14, 'Jolly', '+Velocidade,-Ataque Especial'),
(15, 'Naive', '+Velocidade,-Defesa Especial'),
(16, 'Modest', '+Ataque Especial,-Ataque'),
(17, 'Mild', '+Ataque Especial,-Defesa'),
(18, 'Quiet', '+Ataque Especial,-Velocidade'),
(19, 'Bashful', 'Neutro'),
(20, 'Rash', '+Ataque Especial,-Defesa Especial'),
(21, 'Calm', '+Defesa Especial,-Ataque'),
(22, 'Gentle', '+Defesa Especial,-Defesa'),
(23, 'Sassy', '+Defesa Especial,-Velocidade'),
(24, 'Careful', '+Defesa Especial,-Ataque Especial'),
(25, 'Quirky', 'Neutro');

INSERT INTO Golpe (id_golpe, nome, poder, precisao, id_tipo) VALUES
(1, 'Tackle', 40, 100, 1), -- Normal
(2, 'Vine Whip', 45, 100, 12), -- Grass
(3, 'Ember', 40, 100, 10), -- Fire
(4, 'Water Gun', 40, 100, 11), -- Water
(5, 'Thunder Shock', 40, 100, 13), -- Electric
(6, 'Gust', 40, 100, 3), -- Flying
(7, 'Quick Attack', 40, 100, 1), -- Normal
(8, 'Scratch', 40, 100, 1), -- Normal
(9, 'Razor Leaf', 55, 95, 12), -- Grass
(10, 'Flamethrower', 90, 100, 10), -- Fire
(11, 'Hydro Pump', 110, 80, 11), -- Water
(12, 'Thunderbolt', 90, 100, 13), -- Electric
(13, 'Hyper Beam', 150, 90, 1), -- Normal
(14, 'Earthquake', 100, 100, 5), -- Ground
(15, 'Rock Slide', 75, 90, 6), -- Rock
(16, 'Ice Beam', 90, 100, 15), -- Ice
(17, 'Blizzard', 110, 70, 15), -- Ice
(18, 'Confusion', 50, 100, 14), -- Psychic
(19, 'Psychic', 90, 100, 14), -- Psychic
(20, 'Leech Seed', 0, 90, 12), -- Grass
(21, 'Fire Spin', 35, 85, 10), -- Fire
(22, 'Bubble Beam', 65, 100, 11), -- Water
(23, 'Thunder Wave', 0, 90, 13), -- Electric
(24, 'Poison Sting', 15, 100, 8), -- Poison
(25, 'Acid', 40, 100, 8), -- Poison
(26, 'Sludge', 65, 100, 8), -- Poison
(27, 'Body Slam', 85, 100, 1), -- Normal
(28, 'Double-Edge', 120, 100, 1), -- Normal
(29, 'Wrap', 15, 90, 1), -- Normal
(30, 'Horn Attack', 65, 100, 1), -- Normal
(31, 'Stomp', 65, 100, 1), -- Normal
(32, 'Egg Bomb', 100, 75, 1), -- Normal
(33, 'Peck', 35, 100, 3), -- Flying
(34, 'Drill Peck', 80, 100, 3), -- Flying
(35, 'Aurora Beam', 65, 100, 15), -- Ice
(36, 'Sky Attack', 140, 90, 3), -- Flying
(37, 'Mega Punch', 80, 85, 1), -- Normal
(38, 'Mega Kick', 120, 75, 1), -- Normal
(39, 'Horn Drill', NULL, 30, 1), -- Normal
(40, 'Seismic Toss', NULL, 100, 2), -- Fighting
(41, 'Counter', NULL, 100, 2), -- Fighting
(42, 'Karate Chop', 50, 100, 2), -- Fighting
(43, 'Submission', 80, 80, 2), -- Fighting
(44, 'Low Kick', 50, 90, 2), -- Fighting
(45, 'Night Shade', NULL, 100, 7), -- Ghost
(46, 'Lick', 30, 100, 7), -- Ghost
(47, 'Shadow Punch', 60, 100, 7), -- Ghost
(48, 'Sludge Bomb', 90, 100, 8), -- Poison
(49, 'Dream Eater', 100, 100, 14), -- Psychic
(50, 'Self-Destruct', 200, 100, 1), -- Normal
(51, 'Explosion', 250, 100, 1), -- Normal
(52, 'Spore', NULL, 100, 12), -- Grass
(53, 'Sleep Powder', NULL, 75, 12), -- Grass
(54, 'Stun Spore', NULL, 75, 12), -- Grass
(55, 'Toxic', NULL, 90, 8), -- Poison
(56, 'Haze', NULL, NULL, 15), -- Ice
(57, 'Confuse Ray', NULL, 100, 7), -- Ghost
(58, 'Barrier', NULL, NULL, 14), -- Psychic
(59, 'Light Screen', NULL, NULL, 14), -- Psychic
(60, 'Reflect', NULL, NULL, 14), -- Psychic
(61, 'Swift', 60, NULL, 1), -- Normal
(62, 'Rock Throw', 50, 90, 6), -- Rock
(63, 'Leech Life', 80, 100, 4), -- Bug
(64, 'String Shot', NULL, 95, 4), -- Bug
(65, 'Rage', 20, 100, 1), -- Normal
(66, 'Bide', NULL, NULL, 1), -- Normal
(67, 'Mirror Move', NULL, NULL, 3), -- Flying
(68, 'Transform', NULL, NULL, 1), -- Normal
(69, 'Metronome', NULL, NULL, 1), -- Normal
(70, 'Swords Dance', NULL, NULL, 1), -- Normal
(71, 'Growth', NULL, NULL, 12), -- Grass
(72, 'Harden', NULL, NULL, 1), -- Normal
(73, 'Minimize', NULL, NULL, 1), -- Normal
(74, 'Double Team', NULL, NULL, 1), -- Normal
(75, 'Recover', NULL, NULL, 1), -- Normal
(76, 'Soft-Boiled', NULL, NULL, 1), -- Normal
(77, 'Focus Energy', NULL, NULL, 1), -- Normal
(78, 'Barrage', 15, 85, 1), -- Normal
(79, 'Clamp', 35, 85, 11), -- Water
(80, 'Constrict', 10, 100, 1), -- Normal
(81, 'Dragon Breath', 60, 100, 16); -- Dragon

INSERT INTO Golpe (id_golpe, nome, poder, precisao, id_tipo) VALUES
(104, 'TM09 Take Down', 90, 85, 1),
(122, 'TM23 Dragon Rage', 40, 100, 16),
(123, 'TM25 Thunder', 110, 70, 13),
(127, 'TM28 Dig', 80, 100, 5),
(128, 'TM29 Psychic', 90, 100, 14),
(129, 'TM30 Teleport', NULL, NULL, 14),
(137, 'TM38 Fire Blast', 110, 85, 10),
(143, 'TM44 Rest', NULL, NULL, 1),
(145, 'TM46 Psywave', NULL, 80, 14);

INSERT INTO Insignia (id_insignia, nome, descricao) VALUES
(1, 'Boulder Badge', 'Insígnia da Cidade de Pewter, conquistada ao derrotar Brock.'),
(2, 'Cascade Badge', 'Insígnia da Cidade de Cerulean, conquistada ao derrotar Misty.'),
(3, 'Thunder Badge', 'Insígnia da Cidade de Vermilion, conquistada ao derrotar Lt. Surge.'),
(4, 'Rainbow Badge', 'Insígnia da Cidade de Celadon, conquistada ao derrotar Erika.'),
(5, 'Soul Badge', 'Insígnia da Cidade de Fuchsia, conquistada ao derrotar Koga.'),
(6, 'Marsh Badge', 'Insígnia da Cidade de Saffron, conquistada ao derrotar Sabrina.'),
(7, 'Volcano Badge', 'Insígnia da Ilha Cinnabar, conquistada ao derrotar Blaine.'),
(8, 'Earth Badge', 'Insígnia da Cidade de Viridian, conquistada ao derrotar Giovanni.');

INSERT INTO Ginasio (id_ginasio, id_insignia, nome, cidade) VALUES
(1, 1, 'Ginásio de Pewter', 'Pewter City'),
(2, 2, 'Ginásio de Cerulean', 'Cerulean City'),
(3, 3, 'Ginásio de Vermilion', 'Vermilion City'),
(4, 4, 'Ginásio de Celadon', 'Celadon City'),
(5, 5, 'Ginásio de Fuchsia', 'Fuchsia City'),
(6, 6, 'Ginásio de Saffron', 'Saffron City'),
(7, 7, 'Ginásio de Cinnabar', 'Cinnabar Island'),
(8, 8, 'Ginásio de Viridian', 'Viridian City');

INSERT INTO Treinador (id_treinador, nome, eh_campeao) VALUES
(1, 'Jotave', FALSE),
(2, 'Panza', FALSE),
(3, 'Barbara', FALSE),
(4, 'Daniel', FALSE),
(5, 'Raquel', FALSE),
(6, 'Lorelei', FALSE),
(7, 'Bruno', FALSE),
(8, 'Agatha', FALSE),
(9, 'Lance', FALSE);

INSERT INTO Liga (id_liga, nome) VALUES
(1, 'Liga de Kanto');

INSERT INTO Item (id_item, id_instancia, id_treinador, nome, descricao) VALUES
(1, 1, 5, 'Potion', 'Restaura 20 pontos de vida de um Pokémon.'),
(2, 1, 3, 'Super Potion', 'Restaura 50 pontos de vida de um Pokémon.'),
(3, 1, 1, 'Hyper Potion', 'Restaura 200 pontos de vida de um Pokémon.'),
(4, 1, 2, 'Max Potion', 'Restaura todo o HP de um Pokémon.'),
(5, 1, 4, 'Full Restore', 'Restaura todo o HP e remove todas as condições de status.'),
(6, 1, 4, 'Antidote', 'Cura envenenamento de um Pokémon.'),
(7, 1, 4, 'Paralyze Heal', 'Cura paralisia de um Pokémon.'),
(8, 1, 2, 'Burn Heal', 'Cura queimaduras de um Pokémon.'),
(9, 1, 2, 'Ice Heal', 'Descongela um Pokémon.'),
(10, 1, 1, 'Awakening', 'Acorda um Pokémon adormecido.'),
(11, 1, 5, 'Revive', 'Revive um Pokémon desmaiado com metade do HP.'),
(12, 1, 3, 'Max Revive', 'Revive um Pokémon desmaiado com HP completo.'),
(13, 1, 3, 'Poké Ball', 'Usada para capturar Pokémon selvagens.'),
(13, 2, 5, 'Poké Ball', 'Usada para capturar Pokémon selvagens.'),
(14, 1, 2, 'Great Ball', 'Tem uma taxa de captura melhor que a Poké Ball.'),
(15, 1, 1, 'Ultra Ball', 'Tem uma taxa de captura melhor que a Great Ball.'),
(16, 1, 4, 'Master Ball', 'Captura qualquer Pokémon sem falhar.'),
(17, 1, 2, 'Rare Candy', 'Aumenta o nível de um Pokémon em 1.'),
(18, 1, 1, 'Thunder Stone', 'Evolui Pikachu e Eevee.'),
(19, 1, 2, 'Water Stone', 'Evolui Poliwhirl, Shellder e Eevee.'),
(20, 1, 4, 'Fire Stone', 'Evolui Eevee, Vulpix e Growlithe.'),
(21, 1, 5, 'Leaf Stone', 'Evolui Gloom, Weepinbell e Exeggcute.'),
(22, 1, 3, 'Moon Stone', 'Evolui Clefairy, Nidorina e Nidorino.'),
(23, 1, 4, 'X Attack', 'Aumenta o Ataque em uma batalha.'),
(24, 1, 4, 'X Defense', 'Aumenta a Defesa em uma batalha.'),
(25, 1, 4, 'X Speed', 'Aumenta a Velocidade em uma batalha.'),
(26, 1, 4, 'X Special', 'Aumenta o Ataque Especial em uma batalha.');

INSERT INTO Item_Medicinais (id_item, id_instancia, valor_efeito_medicinal) VALUES
(1, 1, 20),    -- Potion
(2, 1, 50),    -- Super Potion
(3, 1, 200),   -- Hyper Potion
(4, 1, 9999),  -- Max Potion
(5, 1, 9999),  -- Full Restore
(6, 1, 0),     -- Antidote
(7, 1, 0),     -- Paralyze Heal
(8, 1, 0),     -- Burn Heal
(9, 1, 0),     -- Ice Heal
(10, 1, 0),    -- Awakening
(11, 1, 50),   -- Revive
(12, 1, 9999); -- Max Revive

INSERT INTO Item_Pokebola (id_item, id_instancia, taxa_captura) VALUES
(13, 1, 1.0),    -- Poké Ball - Barbara
(13, 2, 1.0),    -- Poké Ball - Raquel
(14, 1, 1.5),    -- Great Ball
(15, 1, 2.0),    -- Ultra Ball
(16, 1, 255.0);  -- Master Ball

INSERT INTO Item_Evolucao (id_item, id_instancia) VALUES
(17, 1),  -- Rare Candy
(18, 1),  -- Thunder Stone
(19, 1),  -- Water Stone
(20, 1),  -- Fire Stone
(21, 1),  -- Leaf Stone
(22, 1);  -- Moon Stone

INSERT INTO Item_Batalha (id_item, id_instancia, valor_efeito_batalha) VALUES
(23, 1, 1),   -- X Attack (Aumenta ataque)
(24, 1, 1),   -- X Defense (Aumenta defesa)
(25, 1, 1),   -- X Speed (Aumenta velocidade)
(26, 1, 1);   -- X Special (Aumenta ataque especial)

-- Inserção de instâncias de Pokémon para Jotave
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza)
VALUES 
(6, 1, 1, 'Charizard', 50, FALSE, 'Saudável', 150, 90, 85, 110, 85, 100, 4),
(25, 1, 1, 'Pikachu', 45, TRUE, 'Saudável', 110, 55, 50, 75, 60, 110, 11),
(2, 1, 1, 'Ivysaur', 45, FALSE, 'Saudável', 130, 80, 90, 95, 85, 70, 5),
(45, 1, 1, 'Vileplume', 40, FALSE, 'Saudável', 140, 90, 100, 110, 90, 50, 8),
(77, 1, 1, 'Ponyta', 42, FALSE, 'Saudável', 105, 85, 80, 75, 60, 90, 2),
(12, 1, 1, 'Butterfree', 48, FALSE, 'Saudável', 100, 60, 70, 90, 70, 85, 3);

-- Inserção de instâncias de Pokémon para Panza
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza)
VALUES 
(149, 1, 2, 'Dragonite', 55, FALSE, 'Saudável', 180, 120, 110, 100, 100, 80, 4),
(3, 1, 2, 'Venusaur', 50, FALSE, 'Saudável', 170, 90, 95, 110, 100, 70, 6),
(20, 1, 2, 'Raticate', 40, FALSE, 'Saudável', 105, 75, 70, 60, 60, 120, 7),
(50, 1, 2, 'Diglett', 45, FALSE, 'Saudável', 75, 80, 60, 50, 40, 130, 2),
(90, 1, 2, 'Shellder', 45, TRUE, 'Saudável', 115, 70, 90, 60, 85, 50, 3),
(130, 1, 2, 'Gyarados', 55, FALSE, 'Saudável', 190, 110, 100, 80, 80, 70, 4);

-- Inserção de instâncias de Pokémon para Barbara (time rosa)
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza)
VALUES 
(36, 1, 3, 'Clefable', 40, FALSE, 'Saudável', 150, 70, 85, 95, 90, 60, 1),
(39, 1, 3, 'Jigglypuff', 38, FALSE, 'Saudável', 120, 60, 60, 70, 70, 50, 5),
(40, 1, 3, 'Wigglytuff', 45, FALSE, 'Saudável', 140, 75, 70, 80, 75, 60, 6),
(113, 1, 3, 'Chansey', 48, FALSE, 'Saudável', 200, 60, 70, 90, 90, 50, 3),
(80, 1, 3, 'Slowbro', 42, FALSE, 'Saudável', 160, 80, 100, 110, 100, 40, 2),
(108, 1, 3, 'Lickitung', 40, FALSE, 'Saudável', 130, 70, 85, 70, 75, 50, 4);

-- Inserção de instâncias de Pokémon para Daniel (competitivamente fortes)
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza)
VALUES 
(65, 1, 4, 'Alakazam', 55, FALSE, 'Saudável', 130, 60, 70, 150, 90, 120, 3),
(130, 2, 4, 'Gyarados', 55, FALSE, 'Saudável', 190, 110, 100, 80, 80, 70, 6),
(143, 1, 4, 'Snorlax', 50, FALSE, 'Saudável', 210, 100, 100, 70, 90, 30, 4),
(135, 1, 4, 'Jolteon', 50, FALSE, 'Saudável', 130, 85, 70, 110, 90, 130, 5),
(94, 1, 4, 'Gengar', 52, FALSE, 'Saudável', 140, 90, 80, 130, 85, 110, 2),
(121, 1, 4, 'Starmie', 50, FALSE, 'Saudável', 150, 80, 90, 120, 100, 100, 3);

-- Inserção de instâncias de Pokémon para Raquel (fracos contra Giovanni)
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza)
VALUES 
(25, 2, 5, 'Pikachu', 30, TRUE, 'Saudável', 100, 55, 50, 75, 60, 110, 11),
(37, 1, 5, 'Vulpix', 38, TRUE, 'Saudável', 95, 70, 65, 75, 60, 90, 2),
(126, 1, 5, 'Magmar', 35, FALSE, 'Saudável', 120, 90, 85, 90, 85, 80, 5),
(109, 1, 5, 'Koffing', 32, FALSE, 'Saudável', 100, 60, 70, 80, 80, 50, 4),
(101, 1, 5, 'Electrode', 30, FALSE, 'Saudável', 100, 70, 60, 80, 60, 150, 3),
(58, 1, 5, 'Growlithe', 38, TRUE, 'Saudável', 105, 85, 80, 75, 60, 90, 2);

-- Inserção de instâncias de Pokémon para Lorelei (especialista em Gelo e Água)
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza)
VALUES 
(87, 1, 6, 'Dewgong', 56, FALSE, 'Saudável', 180, 80, 100, 90, 120, 70, 3),
(91, 1, 6, 'Cloyster', 54, FALSE, 'Saudável', 150, 95, 180, 85, 110, 70, 5),
(80, 2, 6, 'Slowbro', 55, FALSE, 'Saudável', 190, 75, 110, 100, 120, 30, 2),
(124, 1, 6, 'Jynx', 56, FALSE, 'Saudável', 170, 60, 70, 120, 90, 95, 1),
(131, 1, 6, 'Lapras', 56, FALSE, 'Saudável', 200, 85, 90, 95, 110, 80, 6);

-- Inserção de instâncias de Pokémon para Bruno (especialista em Lutador e Rocha)
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza)
VALUES 
(95, 1, 7, 'Onix', 53, FALSE, 'Saudável', 160, 70, 180, 50, 85, 40, 3),
(95, 2, 7, 'Onix', 53, FALSE, 'Saudável', 160, 70, 180, 50, 85, 40, 3),
(66, 1, 7, 'Machop', 55, FALSE, 'Saudável', 180, 100, 85, 70, 85, 60, 2),
(67, 1, 7, 'Machoke', 56, FALSE, 'Saudável', 190, 110, 90, 80, 85, 55, 2),
(68, 1, 7, 'Machamp', 58, FALSE, 'Saudável', 200, 120, 100, 85, 90, 50, 1);

-- Inserção de instâncias de Pokémon para Agatha (especialista em Fantasma e Veneno)
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza)
VALUES 
(94, 2, 8, 'Gengar', 56, FALSE, 'Saudável', 150, 75, 80, 130, 90, 110, 5),
(42, 1, 8, 'Golbat', 55, FALSE, 'Saudável', 170, 85, 90, 80, 80, 100, 4),
(93, 1, 8, 'Haunter', 55, FALSE, 'Saudável', 140, 75, 85, 120, 85, 100, 3),
(24, 1, 8, 'Arbok', 56, FALSE, 'Saudável', 170, 85, 95, 70, 85, 95, 6),
(94, 3, 8, 'Gengar', 58, FALSE, 'Saudável', 160, 80, 85, 140, 100, 120, 2);

-- Inserção de instâncias de Pokémon para Lance (especialista em Dragão)
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza)
VALUES 
(148, 1, 9, 'Dragonair', 56, FALSE, 'Saudável', 170, 95, 100, 85, 85, 90, 5),
(148, 2, 9, 'Dragonair', 56, FALSE, 'Saudável', 170, 85, 85, 95, 100, 90, 4),
(130, 3, 9, 'Gyarados', 58, FALSE, 'Saudável', 200, 120, 100, 90, 85, 80, 3),
(142, 1, 9, 'Aerodactyl', 58, FALSE, 'Saudável', 180, 105, 85, 70, 80, 130, 2),
(149, 2, 9, 'Dragonite', 60, FALSE, 'Saudável', 220, 120, 110, 100, 100, 80, 1);

-- Inserção de instâncias para os Pokémons ainda não inseridos
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza)
VALUES 
(1, 1, NULL, 'Bulbasaur', 5, FALSE, 'Saudável', 20, 50, 50, 65, 60, 45, 1),
(4, 1, 1, 'Charmander', 5, FALSE, 'Saudável', 20, 52, 43, 60, 50, 65, 2),
(5, 1, 1, 'Charmeleon', 5, FALSE, 'Saudável', 20, 52, 43, 60, 50, 65, 2),
(7, 1, NULL, 'Squirtle', 5, FALSE, 'Saudável', 20, 48, 65, 50, 64, 43, 3),
(8, 1, NULL, 'Wartortle', 5, FALSE, 'Saudável', 20, 48, 65, 50, 64, 43, 3),
(9, 1, NULL, 'Blastoise', 5, FALSE, 'Saudável', 20, 83, 100, 85, 105, 78, 2),
(10, 1, NULL, 'Caterpie', 5, FALSE, 'Saudável', 20, 30, 35, 20, 20, 45, 4),
(11, 1, NULL, 'Metapod', 5, FALSE, 'Saudável', 20, 20, 55, 25, 25, 30, 30),
(13, 1, NULL, 'Weedle', 5, FALSE, 'Saudável', 20, 35, 30, 20, 20, 50, 5),
(16, 1, NULL, 'Pidgey', 5, FALSE, 'Saudável', 20, 45, 40, 35, 35, 56, 6),
(19, 1, NULL, 'Rattata', 5, FALSE, 'Saudável', 20, 56, 35, 25, 35, 72, 7),
(21, 1, NULL, 'Spearow', 5, FALSE, 'Saudável', 20, 60, 30, 31, 31, 70, 8),
(23, 1, NULL, 'Ekans', 5, FALSE, 'Saudável', 20, 60, 44, 40, 54, 55, 9),
(27, 1, NULL, 'Sandshrew', 5, FALSE, 'Saudável', 20, 75, 85, 20, 30, 40, 10),
(29, 1, NULL, 'Nidoran♀', 5, FALSE, 'Saudável', 20, 47, 52, 40, 40, 41, 11),
(32, 1, NULL, 'Nidoran♂', 5, FALSE, 'Saudável', 20, 57, 40, 40, 40, 50, 12),
(35, 1, 3, 'Clefairy', 5, TRUE, 'Saudável', 20, 45, 48, 60, 65, 35, 13),
(41, 1, NULL, 'Zubat', 5, FALSE, 'Saudável', 20, 45, 35, 30, 40, 55, 15),
(43, 1, NULL, 'Oddish', 5, FALSE, 'Saudável', 20, 50, 55, 75, 65, 30, 16),
(46, 1, NULL, 'Paras', 5, FALSE, 'Saudável', 20, 70, 55, 45, 55, 25, 17),
(48, 1, NULL, 'Venonat', 5, FALSE, 'Saudável', 20, 55, 50, 40, 55, 45, 18),
(52, 1, NULL, 'Meowth', 5, FALSE, 'Saudável', 20, 45, 35, 40, 40, 90, 20),
(54, 1, NULL, 'Psyduck', 5, FALSE, 'Saudável', 20, 52, 48, 65, 50, 55, 21),
(56, 1, NULL, 'Mankey', 5, FALSE, 'Saudável', 20, 80, 35, 35, 45, 70, 22),
(60, 1, NULL, 'Poliwag', 5, TRUE, 'Saudável', 20, 50, 40, 40, 40, 90, 24),
(63, 1, NULL, 'Abra', 5, FALSE, 'Saudável', 20, 20, 15, 105, 55, 90, 25),
(69, 1, NULL, 'Bellsprout', 5, FALSE, 'Saudável', 20, 75, 35, 70, 30, 40, 24),
(72, 1, NULL, 'Tentacool', 5, FALSE, 'Saudável', 20, 40, 35, 50, 100, 70, 23),
(74, 1, NULL, 'Geodude', 5, FALSE, 'Saudável', 20, 80, 100, 30, 30, 20, 22),
(81, 1, NULL, 'Magnemite', 5, FALSE, 'Saudável', 20, 35, 70, 95, 55, 45, 1),
(84, 1, NULL, 'Doduo', 5, FALSE, 'Saudável', 20, 85, 45, 35, 35, 75, 2),
(86, 1, NULL, 'Seel', 5, FALSE, 'Saudável', 20, 45, 55, 45, 70, 45, 3),
(98, 1, NULL, 'Krabby', 5, FALSE, 'Saudável', 20, 105, 90, 25, 25, 50, 4),
(104, 1, NULL, 'Cubone', 5, FALSE, 'Saudável', 20, 50, 95, 40, 50, 35, 5),
(111, 1, NULL, 'Rhyhorn', 5, FALSE, 'Saudável', 20, 85, 95, 30, 30, 25, 6),
(116, 1, NULL, 'Horsea', 5, FALSE, 'Saudável', 20, 40, 70, 70, 70, 60, 7),
(120, 1, NULL, 'Staryu', 5, FALSE, 'Saudável', 20, 45, 55, 70, 55, 85, 8),
(123, 1, NULL, 'Scyther', 5, FALSE, 'Saudável', 20, 110, 80, 55, 80, 105, 16),
(127, 1, NULL, 'Pinsir', 5, FALSE, 'Saudável', 20, 125, 100, 55, 70, 85, 9),
(137, 1, NULL, 'Porygon', 5, FALSE, 'Saudável', 20, 60, 70, 85, 75, 40, 10),
(138, 1, NULL, 'Omanyte', 5, FALSE, 'Saudável', 20, 40, 100, 90, 55, 35, 18),
(139, 1, NULL, 'Omastar', 5, FALSE, 'Saudável', 20, 60, 125, 115, 70, 55, 19),
(140, 1, NULL, 'Kabuto', 5, FALSE, 'Saudável', 20, 80, 90, 55, 45, 55, 20),
(141, 1, NULL, 'Kabutops', 5, FALSE, 'Saudável', 20, 115, 105, 65, 70, 80, 21),
(144, 1, NULL, 'Articuno', 5, FALSE, 'Saudável', 20, 85, 100, 95, 125, 85, 23),
(145, 1, NULL, 'Zapdos', 5, FALSE, 'Saudável', 20, 90, 85, 125, 90, 100, 24),
(146, 1, NULL, 'Moltres', 5, FALSE, 'Saudável', 20, 100, 90, 125, 85, 90, 25),
(147, 1, 2, 'Dratini', 5, FALSE, 'Saudável', 20, 64, 45, 50, 50, 50, 11),
(150, 1, NULL, 'Mewtwo', 50, FALSE, 'Saudável', 100, 110, 90, 154, 90, 130, 12),
(151, 1, NULL, 'Mew', 50, FALSE, 'Saudável', 100, 100, 100, 100, 100, 100, 13),
(14, 1, NULL, 'Kakuna', 5, FALSE, 'Saudável', 20, 30, 55, 25, 25, 30, 21),
(15, 1, NULL, 'Beedrill', 5, FALSE, 'Saudável', 20, 80, 40, 45, 80, 75, 5),
(17, 1, NULL, 'Pidgeotto', 5, FALSE, 'Saudável', 20, 60, 55, 50, 50, 71, 6),
(18, 1, NULL, 'Pidgeot', 5, FALSE, 'Saudável', 20, 80, 70, 65, 70, 101, 7),
(22, 1, NULL, 'Fearow', 5, FALSE, 'Saudável', 20, 90, 65, 60, 60, 100, 9),
(28, 1, NULL, 'Sandslash', 5, FALSE, 'Saudável', 20, 100, 75, 50, 50, 65, 11),
(31, 1, NULL, 'Nidoqueen', 5, FALSE, 'Saudável', 20, 90, 90, 75, 85, 76, 12),
(34, 1, NULL, 'Nidoking', 5, FALSE, 'Saudável', 20, 102, 77, 85, 75, 85, 13),
(38, 1, NULL, 'Ninetales', 5, FALSE, 'Saudável', 20, 81, 75, 80, 100, 100, 14),
(47, 1, NULL, 'Parasect', 5, FALSE, 'Saudável', 20, 95, 80, 60, 80, 30, 15),
(49, 1, NULL, 'Venomoth', 5, FALSE, 'Saudável', 20, 70, 65, 90, 75, 90, 16),
(51, 1, NULL, 'Dugtrio', 5, FALSE, 'Saudável', 20, 80, 50, 70, 60, 120, 17),
(53, 1, NULL, 'Persian', 5, FALSE, 'Saudável', 20, 70, 60, 65, 65, 115, 18),
(55, 1, NULL, 'Golduck', 5, FALSE, 'Saudável', 20, 82, 78, 95, 80, 85, 19),
(57, 1, NULL, 'Primeape', 5, FALSE, 'Saudável', 20, 105, 60, 70, 70, 95, 20),
(59, 1, NULL, 'Arcanine', 5, FALSE, 'Saudável', 20, 110, 80, 100, 80, 95, 21),
(61, 1, NULL, 'Poliwhirl', 5, TRUE, 'Saudável', 20, 65, 65, 50, 50, 90, 22),
(64, 1, NULL, 'Kadabra', 5, FALSE, 'Saudável', 20, 35, 30, 120, 70, 105, 23),
(70, 1, NULL, 'Weepinbell', 5, FALSE, 'Saudável', 20, 90, 50, 85, 50, 55, 24),
(75, 1, NULL, 'Graveler', 5, FALSE, 'Saudável', 20, 95, 115, 45, 45, 35, 25),
(82, 1, NULL, 'Magneton', 5, FALSE, 'Saudável', 20, 60, 95, 120, 70, 70, 20),
(83, 1, NULL, 'Farfetch’d', 5, FALSE, 'Saudável', 20, 90, 55, 85, 85, 60, 19),
(92, 1, NULL, 'Gastly', 5, FALSE, 'Saudável', 20, 35, 30, 100, 35, 80, 18),
(117, 1, NULL, 'Seadra', 5, FALSE, 'Saudável', 20, 65, 70, 95, 80, 85, 17),
(118, 1, NULL, 'Goldeen', 5, FALSE, 'Saudável', 20, 67, 60, 45, 50, 63, 16),
(26, 1, NULL, 'Raichu', 5, FALSE, 'Saudável', 20, 90, 55, 90, 80, 110, 3),
(30, 1, NULL, 'Nidorina', 5, TRUE, 'Saudável', 20, 70, 70, 55, 55, 56, 4),
(33, 1, NULL, 'Nidorino', 5, TRUE, 'Saudável', 20, 72, 57, 55, 55, 65, 5),
(44, 1, NULL, 'Gloom', 5, FALSE, 'Saudável', 20, 65, 70, 85, 85, 40, 7),
(73, 1, NULL, 'Tentacruel', 5, FALSE, 'Saudável', 20, 70, 65, 120, 100, 100, 10),
(76, 1, NULL, 'Golem', 5, FALSE, 'Saudável', 20, 120, 130, 55, 65, 45, 11),
(79, 1, NULL, 'Slowpoke', 5, FALSE, 'Saudável', 20, 65, 65, 40, 40, 15, 12),
(85, 1, NULL, 'Dodrio', 5, FALSE, 'Saudável', 20, 110, 70, 60, 60, 110, 13),
(88, 1, NULL, 'Grimer', 5, FALSE, 'Saudável', 20, 80, 50, 40, 50, 25, 14),
(89, 1, NULL, 'Muk', 5, FALSE, 'Saudável', 20, 105, 75, 65, 65, 50, 15),
(97, 1, NULL, 'Hypno', 5, FALSE, 'Saudável', 20, 85, 70, 115, 100, 67, 18),
(99, 1, NULL, 'Kingler', 5, FALSE, 'Saudável', 20, 130, 115, 50, 50, 75, 19),
(100, 1, NULL, 'Voltorb', 5, FALSE, 'Saudável', 20, 40, 50, 55, 50, 100, 20),
(102, 1, NULL, 'Exeggcute', 5, TRUE, 'Saudável', 20, 90, 80, 120, 95, 50, 20),
(103, 1, NULL, 'Exeggutor', 5, FALSE, 'Saudável', 20, 95, 85, 125, 100, 55, 21),
(105, 1, NULL, 'Marowak', 5, FALSE, 'Saudável', 20, 80, 110, 50, 80, 45, 22),
(107, 1, NULL, 'Hitmonchan', 5, FALSE, 'Saudável', 20, 105, 79, 110, 80, 76, 23),
(110, 1, NULL, 'Weezing', 5, FALSE, 'Saudável', 20, 90, 120, 85, 85, 60, 24),
(112, 1, NULL, 'Rhydon', 5, FALSE, 'Saudável', 20, 130, 120, 55, 60, 40, 25),
(114, 1, NULL, 'Tangela', 5, FALSE, 'Saudável', 20, 55, 115, 100, 90, 60, 14),
(115, 1, NULL, 'Kangaskhan', 5, FALSE, 'Saudável', 20, 105, 80, 40, 80, 90, 13),
(119, 1, NULL, 'Seaking', 5, FALSE, 'Saudável', 20, 92, 65, 65, 80, 68, 12),
(122, 1, NULL, 'Mr. Mime', 5, FALSE, 'Saudável', 20, 50, 65, 120, 100, 90, 11),
(125, 1, NULL, 'Electabuzz', 5, FALSE, 'Saudável', 20, 83, 75, 95, 85, 105, 10),
(132, 1, NULL, 'Ditto', 5, FALSE, 'Saudável', 20, 48, 48, 48, 48, 48, 9),
(62, 1, NULL, 'Poliwrath ', 5, FALSE, 'Saudável', 20, 48, 48, 48, 48, 48, 8),
(71, 1, NULL, 'Victreebel ', 5, FALSE, 'Saudável', 20, 48, 48, 48, 48, 48, 7),
(78, 1, NULL, 'Rapidash ', 5, FALSE, 'Saudável', 20, 48, 48, 48, 48, 48, 6),
(96, 1, NULL, 'Drowzee', 5, FALSE, 'Saudável', 20, 48, 48, 48, 48, 48, 5),
(106, 1, NULL, 'Hitmonlee', 5, FALSE, 'Saudável', 20, 48, 48, 48, 48, 48, 4),
(128, 1, NULL, 'Tauros', 5, FALSE, 'Saudável', 20, 48, 48, 48, 48, 48, 3),
(129, 1, NULL, 'Magikarp', 5, FALSE, 'Saudável', 20, 48, 48, 48, 48, 48, 2),
(133, 1, NULL, 'Eevee', 5, TRUE, 'Saudável', 20, 48, 48, 48, 48, 48, 1),
(134, 1, NULL, 'Vaporeon', 5, FALSE, 'Saudável', 20, 48, 48, 48, 48, 48, 12),
(136, 1, NULL, 'Flareon', 5, FALSE, 'Saudável', 20, 48, 48, 48, 48, 48, 13),
(148, 3, 2, 'Dragonair', 56, FALSE, 'Saudável', 170, 100, 90, 100, 90, 85, 2);

INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (1, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (1, 1, 2);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (1, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (1, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (1, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (1, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (1, 1, 105);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (1, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (1, 1, 19);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (2, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (2, 1, 2);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (2, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (2, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (2, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (2, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (2, 1, 105);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (2, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (2, 1, 19);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (3, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (3, 1, 2);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (3, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (3, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (3, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (3, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (3, 1, 105);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (3, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (3, 1, 19);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (4, 1, 8);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (4, 1, 3);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (4, 1, 10);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (4, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (4, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (4, 1, 137);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (4, 1, 100);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (5, 1, 8);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (5, 1, 3);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (5, 1, 10);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (5, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (5, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (5, 1, 137);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (5, 1, 100);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (6, 1, 8);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (6, 1, 3);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (6, 1, 10);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (6, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (6, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (6, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (6, 1, 137);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (6, 1, 100);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (7, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (7, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (7, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (7, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (7, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (8, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (8, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (8, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (8, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (8, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (9, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (9, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (9, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (9, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (9, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (10, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (10, 1, 2);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (10, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (10, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (10, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (11, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (11, 1, 2);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (11, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (11, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (11, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (12, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (12, 1, 2);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (12, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (12, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (12, 1, 18);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (12, 1, 19);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (12, 1, 53);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (12, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (13, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (13, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (13, 1, 26);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (13, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (13, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (14, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (14, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (14, 1, 26);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (14, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (14, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (15, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (15, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (15, 1, 26);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (15, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (15, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (16, 1, 6);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (16, 1, 33);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (16, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (16, 1, 34);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (16, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (17, 1, 6);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (17, 1, 33);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (17, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (17, 1, 34);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (17, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (18, 1, 6);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (18, 1, 33);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (18, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (18, 1, 34);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (18, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (19, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (19, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (19, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (19, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (19, 1, 114);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (20, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (20, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (20, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (20, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (20, 1, 114);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (21, 1, 6);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (21, 1, 33);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (21, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (21, 1, 34);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (21, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (22, 1, 6);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (22, 1, 33);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (22, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (22, 1, 34);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (22, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (23, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (23, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (23, 1, 26);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (23, 1, 29);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (23, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (24, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (24, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (24, 1, 26);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (24, 1, 29);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (24, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (25, 1, 5);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (25, 1, 12);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (25, 1, 23);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (25, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (25, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (26, 1, 5);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (26, 1, 12);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (26, 1, 23);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (26, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (26, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (27, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (27, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (27, 1, 31);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (27, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (27, 1, 127);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (28, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (28, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (28, 1, 31);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (28, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (28, 1, 127);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (29, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (29, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (29, 1, 26);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (29, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (29, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (30, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (30, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (30, 1, 26);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (30, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (30, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (31, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (31, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (31, 1, 26);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (31, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (31, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (32, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (32, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (32, 1, 26);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (32, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (32, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (33, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (33, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (33, 1, 26);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (33, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (33, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (34, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (34, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (34, 1, 26);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (34, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (34, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (35, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (35, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (35, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (35, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (35, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (36, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (36, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (36, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (36, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (36, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (37, 1, 3);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (37, 1, 10);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (37, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (37, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (37, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (37, 1, 137);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (38, 1, 3);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (38, 1, 10);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (38, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (38, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (38, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (38, 1, 137);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (39, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (39, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (39, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (39, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (39, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (40, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (40, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (40, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (40, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (40, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (41, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (41, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (41, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (41, 1, 26);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (41, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (42, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (42, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (42, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (42, 1, 26);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (42, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (43, 1, 2);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (43, 1, 12);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (43, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (43, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (43, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (44, 1, 2);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (44, 1, 12);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (44, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (44, 1, 26);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (44, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (44, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (45, 1, 2);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (45, 1, 12);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (45, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (45, 1, 26);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (45, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (45, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (46, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (46, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (46, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (46, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (46, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (47, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (47, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (47, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (47, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (47, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (48, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (48, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (48, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (48, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (48, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (49, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (49, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (49, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (49, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (49, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (50, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (50, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (50, 1, 31);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (50, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (50, 1, 127);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (51, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (51, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (51, 1, 31);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (51, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (51, 1, 127);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (52, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (52, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (52, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (52, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (52, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (53, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (53, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (53, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (53, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (53, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (54, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (54, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (54, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (54, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (54, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (55, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (55, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (55, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (55, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (55, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (56, 1, 2);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (56, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (56, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (56, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (56, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (57, 1, 2);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (57, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (57, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (57, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (57, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (58, 1, 3);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (58, 1, 10);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (58, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (58, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (58, 1, 137);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (59, 1, 3);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (59, 1, 10);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (59, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (59, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (59, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (59, 1, 137);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (60, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (60, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (60, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (60, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (60, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES
(61, 1, 4), (61, 1, 11), (61, 1, 22), (61, 1, 7), (61, 1, 121),
(62, 1, 4), (62, 1, 11), (62, 1, 22), (62, 1, 7), (62, 1, 121),
(63, 1, 18), (63, 1, 19), (63, 1, 7), (63, 1, 14), (63, 1, 128),
(64, 1, 18), (64, 1, 19), (64, 1, 7), (64, 1, 14), (64, 1, 128),
(65, 1, 18), (65, 1, 19), (65, 1, 7), (65, 1, 14), (65, 1, 128),
(66, 1, 2), (66, 1, 7), (66, 1, 27), (66, 1, 30), (66, 1, 104),
(67, 1, 2), (67, 1, 7), (67, 1, 27), (67, 1, 30), (67, 1, 104),
(68, 1, 2), (68, 1, 7), (68, 1, 27), (68, 1, 30), (68, 1, 104),
(69, 1, 2), (69, 1, 12), (69, 1, 20), (69, 1, 7), (69, 1, 121),
(70, 1, 2), (70, 1, 12), (70, 1, 20), (70, 1, 7), (70, 1, 121),
(71, 1, 2), (71, 1, 12), (71, 1, 20), (71, 1, 7), (71, 1, 121),
(72, 1, 8), (72, 1, 11), (72, 1, 22), (72, 1, 7), (72, 1, 121),
(73, 1, 8), (73, 1, 11), (73, 1, 22), (73, 1, 7), (73, 1, 121),
(74, 1, 6), (74, 1, 14), (74, 1, 4), (74, 1, 31), (74, 1, 127),
(75, 1, 6), (75, 1, 14), (75, 1, 4), (75, 1, 31), (75, 1, 127),
(76, 1, 6), (76, 1, 14), (76, 1, 4), (76, 1, 31), (76, 1, 127),
(77, 1, 3), (77, 1, 10), (77, 1, 7), (77, 1, 14), (77, 1, 104), (77, 1, 137),
(78, 1, 3), (78, 1, 10), (78, 1, 7), (78, 1, 14), (78, 1, 104), (78, 1, 137),
(79, 1, 4), (79, 1, 11), (79, 1, 22), (79, 1, 7), (79, 1, 128),
(80, 1, 4), (80, 1, 11), (80, 1, 22), (80, 1, 7), (80, 1, 128),
(81, 1, 13), (81, 1, 7), (81, 1, 5), (81, 1, 12), (81, 1, 123),
(82, 1, 13), (82, 1, 7), (82, 1, 5), (82, 1, 12), (82, 1, 123),
(83, 1, 1), (83, 1, 7), (83, 1, 33), (83, 1, 34), (83, 1, 104),
(84, 1, 6), (84, 1, 33), (84, 1, 7), (84, 1, 34), (84, 1, 104),
(85, 1, 6), (85, 1, 33), (85, 1, 7), (85, 1, 34), (85, 1, 104),
(86, 1, 11), (86, 1, 7), (86, 1, 4), (86, 1, 22), (86, 1, 104),
(87, 1, 11), (87, 1, 7), (87, 1, 4), (87, 1, 22), (87, 1, 104),
(88, 1, 8), (88, 1, 7), (88, 1, 24), (88, 1, 25), (88, 1, 104),
(89, 1, 8), (89, 1, 7), (89, 1, 24), (89, 1, 25), (89, 1, 104),
(90, 1, 11), (90, 1, 7), (90, 1, 4), (90, 1, 22), (90, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (91, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (91, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (91, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (91, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (91, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (92, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (92, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (92, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (92, 1, 128);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (92, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (93, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (93, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (93, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (93, 1, 128);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (93, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (94, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (94, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (94, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (94, 1, 128);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (94, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (95, 1, 6);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (95, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (95, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (95, 1, 127);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (95, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (96, 1, 18);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (96, 1, 19);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (96, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (96, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (96, 1, 128);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (97, 1, 18);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (97, 1, 19);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (97, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (97, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (97, 1, 128);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (98, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (98, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (98, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (98, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (98, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (99, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (99, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (99, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (99, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (99, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (100, 1, 13);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (100, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (100, 1, 5);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (100, 1, 12);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (100, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (101, 1, 13);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (101, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (101, 1, 5);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (101, 1, 12);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (101, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (102, 1, 12);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (102, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (102, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (102, 1, 21);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (102, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (103, 1, 12);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (103, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (103, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (103, 1, 21);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (103, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (104, 1, 5);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (104, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (104, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (104, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (104, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (104, 1, 127);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (105, 1, 5);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (105, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (105, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (105, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (105, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (105, 1, 127);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (106, 1, 2);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (106, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (106, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (106, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (106, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (107, 1, 2);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (107, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (107, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (107, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (107, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (108, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (108, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (108, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (108, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (108, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (109, 1, 8);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (109, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (109, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (109, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (109, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (110, 1, 8);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (110, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (110, 1, 24);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (110, 1, 25);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (110, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (111, 1, 6);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (111, 1, 5);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (111, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (111, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (111, 1, 127);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (112, 1, 6);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (112, 1, 5);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (112, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (112, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (112, 1, 127);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (113, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (113, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (113, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (113, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (113, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (114, 1, 12);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (114, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (114, 1, 20);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (114, 1, 21);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (114, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (115, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (115, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (115, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (115, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (115, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (116, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (116, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (116, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (116, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (116, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (117, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (117, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (117, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (117, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (117, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (118, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (118, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (118, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (118, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (118, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (119, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (119, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (119, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (119, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (119, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (120, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (120, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (120, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (120, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (120, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (121, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (121, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (121, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (121, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (121, 1, 128);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (122, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (122, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (122, 1, 18);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (122, 1, 19);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (122, 1, 128);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (123, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (123, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (123, 1, 33);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (123, 1, 34);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (123, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (124, 1, 15);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (124, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (124, 1, 16);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (124, 1, 17);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (124, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (125, 1, 13);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (125, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (125, 1, 5);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (125, 1, 12);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (125, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (126, 1, 10);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (126, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (126, 1, 3);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (126, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (126, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (126, 1, 137);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (127, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (127, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (127, 1, 33);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (127, 1, 34);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (127, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (128, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (128, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (128, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (128, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (128, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (129, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (129, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (129, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (129, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (129, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (130, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (130, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (130, 1, 6);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (130, 1, 33);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (130, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (131, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (131, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (131, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (131, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (131, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (132, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (132, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (132, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (132, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (132, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (133, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (133, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (133, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (133, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (133, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (134, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (134, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (134, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (134, 1, 22);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (134, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (135, 1, 13);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (135, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (135, 1, 5);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (135, 1, 12);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (135, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (136, 1, 10);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (136, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (136, 1, 3);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (136, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (136, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (137, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (137, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (137, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (137, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (137, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (138, 1, 6);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (138, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (138, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (138, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (138, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (139, 1, 6);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (139, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (139, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (139, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (139, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (140, 1, 6);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (140, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (140, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (140, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (140, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (141, 1, 6);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (141, 1, 11);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (141, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (141, 1, 4);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (141, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (142, 1, 6);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (142, 1, 3);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (142, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (142, 1, 34);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (142, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (143, 1, 1);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (143, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (143, 1, 27);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (143, 1, 30);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (143, 1, 143);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (144, 1, 15);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (144, 1, 3);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (144, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (144, 1, 16);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (144, 1, 17);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (145, 1, 13);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (145, 1, 3);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (145, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (145, 1, 5);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (145, 1, 12);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (145, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (146, 1, 10);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (146, 1, 3);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (146, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (146, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (146, 1, 104);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (146, 1, 137);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (147, 1, 16);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (147, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (147, 1, 81);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (147, 1, 122);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (147, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (147, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (148, 1, 16);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (148, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (148, 1, 81);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (148, 1, 122);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (148, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (148, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (149, 1, 16);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (149, 1, 3);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (149, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (149, 1, 81);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (149, 1, 122);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (149, 1, 123);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (149, 1, 121);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (150, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (150, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (150, 1, 18);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (150, 1, 19);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (150, 1, 128);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (150, 1, 145);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (151, 1, 14);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (151, 1, 7);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (151, 1, 18);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (151, 1, 19);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (151, 1, 128);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (151, 1, 137);
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpe) VALUES (151, 1, 145);

INSERT INTO Equipe (id_equipe, id_treinador) VALUES
(1, 1);

INSERT INTO Equipe (id_equipe, id_treinador) VALUES
(2, 2);

INSERT INTO Equipe (id_equipe, id_treinador) VALUES
(3, 3); 

INSERT INTO Equipe (id_equipe, id_treinador) VALUES
(4, 4);

INSERT INTO Equipe (id_equipe, id_treinador) VALUES
(5, 5);

INSERT INTO Equipe (id_equipe, id_treinador) VALUES
(6, 6);  

INSERT INTO Equipe (id_equipe, id_treinador) VALUES
(7, 7);  

INSERT INTO Equipe (id_equipe, id_treinador) VALUES
(8, 8);  

INSERT INTO Equipe (id_equipe, id_treinador) VALUES
(9, 9);  

INSERT INTO Pokemon_eh_de_tal (id_pokedex_pokemon, id_tipo)
VALUES
-- Bulbasaur
(1, 12), -- Grass
(1, 8),  -- Poison

-- Ivysaur
(2, 12), -- Grass
(2, 8),  -- Poison

-- Venusaur
(3, 12), -- Grass
(3, 8),  -- Poison

-- Charmander
(4, 10), -- Fire

-- Charmeleon
(5, 10), -- Fire

-- Charizard
(6, 10), -- Fire
(6, 3),  -- Flying

-- Squirtle
(7, 11), -- Water

-- Wartortle
(8, 11), -- Water

-- Blastoise
(9, 11), -- Water

-- Caterpie
(10, 4), -- Bug

-- Metapod
(11, 4), -- Bug

-- Butterfree
(12, 4), -- Bug
(12, 3), -- Flying

-- Weedle
(13, 4), -- Bug
(13, 8), -- Poison

-- Kakuna
(14, 4), -- Bug
(14, 8), -- Poison

-- Beedrill
(15, 4), -- Bug
(15, 8), -- Poison

-- Pidgey
(16, 3), -- Flying
(16, 1), -- Normal

-- Pidgeotto
(17, 3), -- Flying
(17, 1), -- Normal

-- Pidgeot
(18, 3), -- Flying
(18, 1), -- Normal

-- Rattata
(19, 1), -- Normal

-- Raticate
(20, 1), -- Normal

-- Spearow
(21, 3), -- Flying
(21, 1), -- Normal

-- Fearow
(22, 3), -- Flying
(22, 1), -- Normal

-- Ekans
(23, 8), -- Poison

-- Arbok
(24, 8), -- Poison

-- Pikachu
(25, 13), -- Electric

-- Raichu
(26, 13), -- Electric

-- Sandshrew
(27, 5), -- Ground

-- Sandslash
(28, 5), -- Ground

-- Nidoran♀
(29, 8), -- Poison

-- Nidorina
(30, 8), -- Poison

-- Nidoqueen
(31, 8), -- Poison
(31, 5), -- Ground

-- Nidoran♂
(32, 8), -- Poison

-- Nidorino
(33, 8), -- Poison

-- Nidoking
(34, 8), -- Poison
(34, 5), -- Ground

-- Clefairy
(35, 1), -- Normal

-- Clefable
(36, 1), -- Normal

-- Vulpix
(37, 10), -- Fire

-- Ninetales
(38, 10), -- Fire

-- Jigglypuff
(39, 1),  -- Normal

-- Wigglytuff
(40, 1),  -- Normal

-- Zubat
(41, 8), -- Poison
(41, 3), -- Flying

-- Golbat
(42, 8), -- Poison
(42, 3), -- Flying

-- Oddish
(43, 12), -- Grass
(43, 8),  -- Poison

-- Gloom
(44, 12), -- Grass
(44, 8),  -- Poison

-- Vileplume
(45, 12), -- Grass
(45, 8),  -- Poison

-- Paras
(46, 4), -- Bug
(46, 12), -- Grass

-- Parasect
(47, 4), -- Bug
(47, 12), -- Grass

-- Venonat
(48, 4), -- Bug
(48, 8), -- Poison

-- Venomoth
(49, 4), -- Bug
(49, 8), -- Poison

-- Diglett
(50, 5), -- Ground

-- Dugtrio
(51, 5), -- Ground

-- Meowth
(52, 1), -- Normal

-- Persian
(53, 1), -- Normal

-- Psyduck
(54, 11), -- Water

-- Golduck
(55, 11), -- Water

-- Mankey
(56, 2), -- Fighting

-- Primeape
(57, 2), -- Fighting

-- Growlithe
(58, 10), -- Fire

-- Arcanine
(59, 10), -- Fire

-- Poliwag
(60, 11), -- Water

-- Poliwhirl
(61, 11), -- Water

-- Poliwrath
(62, 11), -- Water
(62, 2), -- Fighting

-- Abra
(63, 14), -- Psychic

-- Kadabra
(64, 14), -- Psychic

-- Alakazam
(65, 14), -- Psychic

-- Machop
(66, 2), -- Fighting

-- Machoke
(67, 2), -- Fighting

-- Machamp
(68, 2), -- Fighting

-- Bellsprout
(69, 12), -- Grass
(69, 8),  -- Poison

-- Weepinbell
(70, 12), -- Grass
(70, 8),  -- Poison

-- Victreebel
(71, 12), -- Grass
(71, 8),  -- Poison

-- Tentacool
(72, 11), -- Water
(72, 8),  -- Poison

-- Tentacruel
(73, 11), -- Water
(73, 8),  -- Poison

-- Geodude
(74, 6), -- Rock
(74, 5), -- Ground

-- Graveler
(75, 6), -- Rock
(75, 5), -- Ground

-- Golem
(76, 6), -- Rock
(76, 5), -- Ground

-- Ponyta
(77, 10), -- Fire

-- Rapidash
(78, 10), -- Fire

-- Slowpoke
(79, 11), -- Water
(79, 14), -- Psychic

-- Slowbro
(80, 11), -- Water
(80, 14), -- Psychic

-- Magnemite
(81, 13), -- Electric

-- Magneton
(82, 13), -- Electric

-- Farfetch'd
(83, 1), -- Normal
(83, 3), -- Flying

-- Doduo
(84, 1), -- Normal
(84, 3), -- Flying

-- Dodrio
(85, 1), -- Normal
(85, 3), -- Flying

-- Seel
(86, 11), -- Water

-- Dewgong
(87, 11), -- Water
(87, 15), -- Ice

-- Grimer
(88, 8), -- Poison

-- Muk
(89, 8), -- Poison

-- Shellder
(90, 11), -- Water

-- Cloyster
(91, 11), -- Water
(91, 15), -- Ice

-- Gastly
(92, 7), -- Ghost
(92, 8), -- Poison

-- Haunter
(93, 7), -- Ghost
(93, 8), -- Poison

-- Gengar
(94, 7), -- Ghost
(94, 8), -- Poison

-- Onix
(95, 6), -- Rock
(95, 5), -- Ground

-- Drowzee
(96, 14), -- Psychic

-- Hypno
(97, 14), -- Psychic

-- Krabby
(98, 11), -- Water

-- Kingler
(99, 11), -- Water

-- Voltorb
(100, 13), -- Electric

-- Electrode
(101, 13), -- Electric

-- Exeggcute
(102, 12), -- Grass
(102, 14), -- Psychic

-- Exeggutor
(103, 12), -- Grass
(103, 14), -- Psychic

-- Cubone
(104, 5), -- Ground

-- Marowak
(105, 5), -- Ground

-- Hitmonlee
(106, 2), -- Fighting

-- Hitmonchan
(107, 2), -- Fighting

-- Lickitung
(108, 1), -- Normal

-- Koffing
(109, 8), -- Poison

-- Weezing
(110, 8), -- Poison

-- Rhyhorn
(111, 5), -- Ground
(111, 6), -- Rock

-- Rhydon
(112, 5), -- Ground
(112, 6), -- Rock

-- Chansey
(113, 1), -- Normal

-- Tangela
(114, 12), -- Grass

-- Kangaskhan
(115, 1), -- Normal

-- Horsea
(116, 11), -- Water

-- Seadra
(117, 11), -- Water

-- Goldeen
(118, 11), -- Water

-- Seaking
(119, 11), -- Water

-- Staryu
(120, 11), -- Water

-- Starmie
(121, 11), -- Water
(121, 14), -- Psychic

-- Mr. Mime
(122, 14), -- Psychic

-- Scyther
(123, 4), -- Bug
(123, 3), -- Flying

-- Jynx
(124, 15), -- Ice
(124, 14), -- Psychic

-- Electabuzz
(125, 13), -- Electric

-- Magmar
(126, 10), -- Fire

-- Pinsir
(127, 4), -- Bug

-- Tauros
(128, 1), -- Normal

-- Magikarp
(129, 11), -- Water

-- Gyarados
(130, 11), -- Water
(130, 3), -- Flying

-- Lapras
(131, 11), -- Water
(131, 15), -- Ice

-- Ditto
(132, 1), -- Normal

-- Eevee
(133, 1), -- Normal

-- Vaporeon
(134, 11), -- Water

-- Jolteon
(135, 13), -- Electric

-- Flareon
(136, 10), -- Fire

-- Porygon
(137, 1), -- Normal

-- Omanyte
(138, 6), -- Rock
(138, 11), -- Water

-- Omastar
(139, 6), -- Rock
(139, 11), -- Water

-- Kabuto
(140, 6), -- Rock
(140, 11), -- Water

-- Kabutops
(141, 6), -- Rock
(141, 11), -- Water

-- Aerodactyl
(142, 6), -- Rock
(142, 3), -- Flying

-- Snorlax
(143, 1), -- Normal

-- Articuno
(144, 15), -- Ice
(144, 3), -- Flying

-- Zapdos
(145, 13), -- Electric
(145, 3), -- Flying

-- Moltres
(146, 10), -- Fire
(146, 3), -- Flying

-- Dratini
(147, 16), -- Dragon

-- Dragonair
(148, 16), -- Dragon

-- Dragonite
(149, 16), -- Dragon
(149, 3),  -- Flying

-- Mewtwo
(150, 14), -- Psychic

-- Mew
(151, 14); -- Psychic

INSERT INTO Item_eh_de_tal (id_item, id_instancia, id_tipo)
VALUES
(18, 1, 13), -- Thunder Stone (Electric)
(19, 1, 11), -- Water Stone (Water)
(20, 1, 10), -- Fire Stone (Fire)
(21, 1, 12), -- Leaf Stone (Grass)
(22, 1, 1), -- Moon Stone (Normal)
(22, 1, 8); -- Moon Stone (Poison)

INSERT INTO Eh_formada (id_liga, id_treinador)
VALUES
(1, 6), -- Lorelei
(1, 7), -- Bruno
(1, 8), -- Agatha
(1, 9); -- Lance

INSERT INTO Possui (id_treinador, id_insignia)
VALUES
(1, 1),
(1, 2),
(1, 3),
(1, 4),
(1, 5),
(1, 6),
(1, 7),
(1, 8),
(2, 1),
(2, 2),
(2, 3),
(2, 4),
(2, 5),
(2, 6),
(2, 7),
(2, 8),
(3, 1),
(3, 2),
(3, 3),
(3, 4),
(3, 5),
(3, 6),
(3, 7),
(3, 8),
(4, 1),
(4, 2),
(4, 3),
(4, 4),
(4, 5),
(4, 6),
(4, 7),
(4, 8),
(5, 1),
(5, 2),
(5, 3),
(5, 4),
(5, 5),
(5, 6),
(5, 7);

INSERT INTO Eh_composta (id_equipe, id_pokedex_pokemon, id_instancia_pokemon)
VALUES
-- Equipe 1
(1, 6, 1),  -- Charizard
(1, 25, 1), -- Pikachu
(1, 2, 1),  -- Ivysaur
(1, 45, 1), -- Vileplume
(1, 77, 1), -- Ponyta
(1, 12, 1), -- Butterfree

-- Equipe 2
(2, 149, 1), -- Dragonite
(2, 3, 1),   -- Venusaur
(2, 20, 1),  -- Raticate
(2, 50, 1),  -- Diglett
(2, 90, 1),  -- Shellder
(2, 130, 1), -- Gyarados

-- Equipe 3
(3, 36, 1),  -- Clefable
(3, 39, 1),  -- Jigglypuff
(3, 40, 1),  -- Wigglytuff
(3, 113, 1), -- Chansey
(3, 80, 1),  -- Slowbro
(3, 108, 1), -- Lickitung

-- Equipe 4
(4, 65, 1),   -- Alakazam
(4, 130, 2),  -- Gyarados
(4, 143, 1),  -- Snorlax
(4, 135, 1),  -- Jolteon
(4, 94, 1),   -- Gengar
(4, 121, 1),  -- Starmie

-- Equipe 5
(5, 25, 2),  -- Pikachu
(5, 37, 1),  -- Vulpix
(5, 126, 1), -- Magmar
(5, 109, 1), -- Koffing
(5, 101, 1), -- Electrode
(5, 58, 1),  -- Growlithe

-- Lorelei
(6, 87, 1),  -- Dewgong
(6, 91, 1),  -- Cloyster
(6, 80, 2),  -- Slowbro
(6, 124, 1), -- Jynx
(6, 131, 1), -- Lapras

-- Bruno
(7, 95, 1),  -- Onix
(7, 95, 2),  -- Onix
(7, 66, 1),  -- Machop
(7, 67, 1),  -- Machoke
(7, 68, 1),  -- Machamp

-- Agatha
(8, 94, 2),  -- Gengar
(8, 42, 1),  -- Golbat
(8, 93, 1),  -- Haunter
(8, 24, 1),  -- Arbok
(8, 94, 3),  -- Gengar

-- Lance
(9, 148, 1), -- Dragonair
(9, 148, 2), -- Dragonair
(9, 130, 3), -- Gyarados
(9, 142, 1), -- Aerodactyl
(9, 149, 2); -- Dragonite

-- Inserção de golpes para os Pokémons de Panza
INSERT INTO Sabe (id_pokedex_pokemon, id_instancia_pokemon, id_golpe)
VALUES
(149, 1, 81), -- Dragonite: Dragon Breath
(149, 1, 16), -- Dragonite: Ice Beam
(149, 1, 12), -- Dragonite: Thunderbolt
(149, 1, 10), -- Dragonite: Flamethrower

(3, 1, 9), -- Venusaur: Razor Leaf
(3, 1, 19), -- Venusaur: Psychic
(3, 1, 20), -- Venusaur: Leech Seed
(3, 1, 18), -- Venusaur: Confusion

(20, 1, 7), -- Raticate: Quick Attack
(20, 1, 27), -- Raticate: Body Slam
(20, 1, 13), -- Raticate: Hyper Beam
(20, 1, 1), -- Raticate: Tackle

(50, 1, 14), -- Diglett: Earthquake
(50, 1, 62), -- Diglett: Rock Throw
(50, 1, 7), -- Diglett: Quick Attack
(50, 1, 80), -- Diglett: Constrict

(90, 1, 11), -- Shellder: Hydro Pump
(90, 1, 22), -- Shellder: Bubble Beam
(90, 1, 16), -- Shellder: Ice Beam
(90, 1, 17), -- Shellder: Blizzard

(130, 1, 14), -- Gyarados: Earthquake
(130, 1, 12), -- Gyarados: Thunderbolt
(130, 1, 4), -- Gyarados: Water Gun
(130, 1, 22); -- Gyarados: Bubble Beam

-- Inserção de golpes para os Pokémons de Jotave
INSERT INTO Sabe (id_pokedex_pokemon, id_instancia_pokemon, id_golpe)
VALUES
-- Charizard
(6, 1, 3),  -- Ember
(6, 1, 10), -- Flamethrower
(6, 1, 137), -- Fire Blast
(6, 1, 14), -- Earthquake

-- Pikachu
(25, 1, 5),  -- Thunder Shock
(25, 1, 12), -- Thunderbolt
(25, 1, 123), -- Thunder
(25, 1, 23), -- Thunder Wave

-- Ivysaur
(2, 1, 2),  -- Vine Whip
(2, 1, 9),  -- Razor Leaf
(2, 1, 20), -- Leech Seed
(2, 1, 19), -- Psychic

-- Vileplume
(45, 1, 2),  -- Vine Whip
(45, 1, 9),  -- Razor Leaf
(45, 1, 20), -- Leech Seed
(45, 1, 26), -- Sludge

-- Ponyta
(77, 1, 3),  -- Ember
(77, 1, 10), -- Flamethrower
(77, 1, 137), -- Fire Blast
(77, 1, 14), -- Earthquake

-- Butterfree
(12, 1, 18), -- Confusion
(12, 1, 19), -- Psychic
(12, 1, 53), -- Sleep Powder
(12, 1, 20); -- Leech Seed

-- Inserção de golpes para os Pokémons de Barbara
INSERT INTO Sabe (id_pokedex_pokemon, id_instancia_pokemon, id_golpe)
VALUES
(36, 1, 1), -- Clefable: Tackle
(36, 1, 19), -- Clefable: Psychic
(36, 1, 12), -- Clefable: Thunderbolt
(36, 1, 27), -- Clefable: Body Slam

(39, 1, 1), -- Jigglypuff: Tackle
(39, 1, 27), -- Jigglypuff: Body Slam
(39, 1, 3), -- Jigglypuff: Ember
(39, 1, 4), -- Jigglypuff: Water Gun

(40, 1, 7), -- Wigglytuff: Quick Attack
(40, 1, 27), -- Wigglytuff: Body Slam
(40, 1, 12), -- Wigglytuff: Thunderbolt
(40, 1, 10), -- Wigglytuff: Flamethrower

(113, 1, 7), -- Chansey: Quick Attack
(113, 1, 18), -- Chansey: Confusion
(113, 1, 19), -- Chansey: Psychic
(113, 1, 76), -- Chansey: Soft-Boiled

(80, 1, 4), -- Slowbro: Water Gun
(80, 1, 19), -- Slowbro: Psychic
(80, 1, 18), -- Slowbro: Confusion
(80, 1, 58), -- Slowbro: Barrier

(108, 1, 1), -- Lickitung: Tackle
(108, 1, 27), -- Lickitung: Body Slam
(108, 1, 28), -- Lickitung: Double-Edge
(108, 1, 10); -- Lickitung: Flamethrower

-- Inserção de golpes para os Pokémons de Daniel
INSERT INTO Sabe (id_pokedex_pokemon, id_instancia_pokemon, id_golpe)
VALUES
(65, 1, 19), -- Alakazam: Psychic
(65, 1, 18), -- Alakazam: Confusion
(65, 1, 58), -- Alakazam: Barrier
(65, 1, 59), -- Alakazam: Light Screen

(130, 2, 14), -- Gyarados: Earthquake
(130, 2, 12), -- Gyarados: Thunderbolt
(130, 2, 4), -- Gyarados: Water Gun
(130, 2, 22), -- Gyarados: Bubble Beam

(143, 1, 27), -- Snorlax: Body Slam
(143, 1, 13), -- Snorlax: Hyper Beam
(143, 1, 143), -- Snorlax: Rest
(143, 1, 1), -- Snorlax: Tackle

(135, 1, 12), -- Jolteon: Thunderbolt
(135, 1, 5), -- Jolteon: Thunder Shock
(135, 1, 23), -- Jolteon: Thunder Wave
(135, 1, 123), -- Jolteon: Thunder

(94, 1, 45), -- Gengar: Night Shade
(94, 1, 48), -- Gengar: Sludge Bomb
(94, 1, 47), -- Gengar: Shadow Punch
(94, 1, 19), -- Gengar: Psychic

(121, 1, 4), -- Starmie: Water Gun
(121, 1, 11), -- Starmie: Hydro Pump
(121, 1, 19), -- Starmie: Psychic
(121, 1, 12); -- Starmie: Thunderbolt

-- Inserção de golpes para os Pokémons de Raquel
INSERT INTO Sabe (id_pokedex_pokemon, id_instancia_pokemon, id_golpe)
VALUES
(25, 2, 5), -- Pikachu: Thunder Shock
(25, 2, 12), -- Pikachu: Thunderbolt
(25, 2, 123), -- Pikachu: Thunder
(25, 2, 23), -- Pikachu: Thunder Wave

(37, 1, 3), -- Vulpix: Ember
(37, 1, 10), -- Vulpix: Flamethrower
(37, 1, 21), -- Vulpix: Fire Spin
(37, 1, 137), -- Vulpix: Fire Blast

(126, 1, 3), -- Magmar: Ember
(126, 1, 10), -- Magmar: Flamethrower
(126, 1, 21), -- Magmar: Fire Spin
(126, 1, 137), -- Magmar: Fire Blast

(109, 1, 24), -- Koffing: Poison Sting
(109, 1, 25), -- Koffing: Acid
(109, 1, 26), -- Koffing: Sludge
(109, 1, 55), -- Koffing: Toxic

(101, 1, 12), -- Electrode: Thunderbolt
(101, 1, 5), -- Electrode: Thunder Shock
(101, 1, 23), -- Electrode: Thunder Wave
(101, 1, 123), -- Electrode: Thunder

(58, 1, 3), -- Growlithe: Ember
(58, 1, 10), -- Growlithe: Flamethrower
(58, 1, 21), -- Growlithe: Fire Spin
(58, 1, 137); -- Growlithe: Fire Blast

-- Lorelei
INSERT INTO Sabe (id_pokedex_pokemon, id_instancia_pokemon, id_golpe)
VALUES
(87, 1, 16), -- Dewgong: Ice Beam
(87, 1, 17), -- Dewgong: Blizzard
(87, 1, 4),  -- Dewgong: Water Gun
(87, 1, 19), -- Dewgong: Psychic

(91, 1, 16), -- Cloyster: Ice Beam
(91, 1, 17), -- Cloyster: Blizzard
(91, 1, 14), -- Cloyster: Earthquake
(91, 1, 22), -- Cloyster: Bubble Beam

(80, 2, 19), -- Slowbro: Psychic
(80, 2, 4),  -- Slowbro: Water Gun
(80, 2, 22), -- Slowbro: Bubble Beam
(80, 2, 58), -- Slowbro: Barrier

(124, 1, 16), -- Jynx: Ice Beam
(124, 1, 19), -- Jynx: Psychic
(124, 1, 17), -- Jynx: Blizzard
(124, 1, 57), -- Jynx: Confuse Ray

(131, 1, 16), -- Lapras: Ice Beam
(131, 1, 17), -- Lapras: Blizzard
(131, 1, 22), -- Lapras: Bubble Beam
(131, 1, 4);  -- Lapras: Water Gun

-- Bruno
INSERT INTO Sabe (id_pokedex_pokemon, id_instancia_pokemon, id_golpe)
VALUES
(95, 1, 14), -- Onix: Earthquake
(95, 1, 15), -- Onix: Rock Slide
(95, 1, 62), -- Onix: Rock Throw
(95, 1, 4),  -- Onix: Water Gun

(95, 2, 14), -- Onix: Earthquake
(95, 2, 15), -- Onix: Rock Slide
(95, 2, 62), -- Onix: Rock Throw
(95, 2, 4),  -- Onix: Water Gun

(66, 1, 43), -- Machop: Submission
(66, 1, 42), -- Machop: Karate Chop
(66, 1, 44), -- Machop: Low Kick
(66, 1, 40), -- Machop: Seismic Toss

(67, 1, 43), -- Machoke: Submission
(67, 1, 42), -- Machoke: Karate Chop
(67, 1, 44), -- Machoke: Low Kick
(67, 1, 40), -- Machoke: Seismic Toss

(68, 1, 43), -- Machamp: Submission
(68, 1, 42), -- Machamp: Karate Chop
(68, 1, 44), -- Machamp: Low Kick
(68, 1, 40); -- Machamp: Seismic Toss

-- Agatha
INSERT INTO Sabe (id_pokedex_pokemon, id_instancia_pokemon, id_golpe)
VALUES
(94, 2, 45), -- Gengar: Night Shade
(94, 2, 48), -- Gengar: Sludge Bomb
(94, 2, 47), -- Gengar: Shadow Punch
(94, 2, 49), -- Gengar: Dream Eater

(42, 1, 6),  -- Golbat: Gust
(42, 1, 24), -- Golbat: Poison Sting
(42, 1, 33), -- Golbat: Peck
(42, 1, 57), -- Golbat: Confuse Ray

(93, 1, 45), -- Haunter: Night Shade
(93, 1, 48), -- Haunter: Sludge Bomb
(93, 1, 47), -- Haunter: Shadow Punch
(93, 1, 57), -- Haunter: Confuse Ray

(24, 1, 24), -- Arbok: Poison Sting
(24, 1, 25), -- Arbok: Acid
(24, 1, 26), -- Arbok: Sludge
(24, 1, 14), -- Arbok: Earthquake

(94, 3, 45), -- Gengar: Night Shade
(94, 3, 48), -- Gengar: Sludge Bomb
(94, 3, 47), -- Gengar: Shadow Punch
(94, 3, 49); -- Gengar: Dream Eater

-- Lance
INSERT INTO Sabe (id_pokedex_pokemon, id_instancia_pokemon, id_golpe)
VALUES
(148, 1, 81), -- Dragonair: Dragon Breath
(148, 1, 14), -- Dragonair: Earthquake
(148, 1, 4),  -- Dragonair: Water Gun
(148, 1, 12), -- Dragonair: Thunderbolt

(148, 2, 81), -- Dragonair: Dragon Breath
(148, 2, 14), -- Dragonair: Earthquake
(148, 2, 4),  -- Dragonair: Water Gun
(148, 2, 12), -- Dragonair: Thunderbolt

(130, 3, 14), -- Gyarados: Earthquake
(130, 3, 12), -- Gyarados: Thunderbolt
(130, 3, 4),  -- Gyarados: Water Gun
(130, 3, 22), -- Gyarados: Bubble Beam

(142, 1, 6),  -- Aerodactyl: Gust
(142, 1, 15), -- Aerodactyl: Rock Slide
(142, 1, 62), -- Aerodactyl: Rock Throw
(142, 1, 81), -- Aerodactyl: Dragon Breath

(149, 2, 81), -- Dragonite: Dragon Breath
(149, 2, 16), -- Dragonite: Ice Beam
(149, 2, 12), -- Dragonite: Thunderbolt
(149, 2, 10); -- Dragonite: Flamethrower

-- Batalhas dos treinadores contra os membros da liga
INSERT INTO Batalha (id_treinador_1, id_treinador_2, vencedor) VALUES
(1, 6, 1),   -- Jotave vs Lorelei
(1, 7, 7),   -- Jotave vs Bruno

(2, 6, 2),   -- Panza vs Lorelei
(2, 7, 2),   -- Panza vs Bruno
(2, 8, 2),   -- Panza vs Agatha
(2, 9, 9),   -- Panza vs Lance

(3, 6, 3),   -- Barbara vs Lorelei
(3, 7, 3),   -- Barbara vs Bruno
(3, 8, 8),   -- Barbara vs Agatha

(4, 6, 4),   -- Daniel vs Lorelei
(4, 7, 4),   -- Daniel vs Bruno
(4, 8, 4),   -- Daniel vs Agatha
(4, 9, 4);   -- Daniel vs Lance

-- Batalhas dos 5 treinadores batalhando entre eles
INSERT INTO Batalha (id_treinador_1, id_treinador_2, vencedor) VALUES
(1, 2, 1),    -- Jotave vs Panza
(2, 3, 3),    -- Panza vs Barbara
(3, 4, 4),    -- Barbara vs Daniel
(4, 5, 4),    -- Daniel vs Raquel
(5, 1, 1);    -- Raquel vs Jotave
"""
]

# Executando os comandos
for command in sql_commands:
    cursor.execute(command)
    
print("Inserções criadas com sucesso!")

Inserções criadas com sucesso!


In [11]:
# Consultas no banco de dados
sql_commands = [
"""
-- Select 1 - Mostra quantas vitorias em batalhas os 10 melhores treinadores tem
SELECT 
    t.nome AS vencedor, 
    COUNT(*) AS vitorias 
FROM 
    Batalha b 
JOIN 
    Treinador t 
ON 
    b.vencedor = t.id_treinador
GROUP BY 
    t.nome
ORDER BY 
    vitorias DESC
LIMIT 10;
""",
"""
-- Select 2 - Mostra quais pokemons dos treinadores tem ataque ou ataque especial maior do que 100
SELECT 
    t.nome AS treinador, 
    STRING_AGG(p.nome, ', ') AS pokemons,
    ARRAY_AGG(p.ataque) AS ataques,
    ARRAY_AGG(p.ataque_especial) AS ataques_especiais
FROM 
    Treinador t
JOIN 
    Equipe e ON e.id_treinador = t.id_treinador
JOIN 
    Eh_composta ec ON ec.id_equipe = e.id_equipe
JOIN 
    Pokemon p ON p.id_pokedex_pokemon = ec.id_pokedex_pokemon 
              AND p.id_instancia_pokemon = ec.id_instancia_pokemon
WHERE 
    p.ataque >= 100 OR p.ataque_especial >= 100
GROUP BY 
    t.nome;
""",
"""  
-- Select 3 - Mostra quantos pokemons tem de cada combinação de tipos (essa combinação sendo 1 ou mais tipos)
WITH PokedexTipos AS (
    SELECT 
        p.id_pokedex,
        CASE 
            WHEN COUNT(pt.id_tipo) > 1 THEN STRING_AGG(t.nome, ', ')
            ELSE STRING_AGG(t.nome, ', ')
        END AS tipos
    FROM 
        Pokedex p
    JOIN 
        Pokemon_eh_de_tal pt ON pt.id_pokedex_pokemon = p.id_pokedex
    JOIN 
        Tipo t ON t.id_tipo = pt.id_tipo
    GROUP BY 
        p.id_pokedex
)
SELECT 
    tipos, 
    COUNT(*) AS quantidade
FROM 
    PokedexTipos
GROUP BY 
    tipos
ORDER BY 
    quantidade DESC;

""",
"""
-- Select 4 - Mostra quantos golpes de cada tipo tem
SELECT 
    t.nome AS tipo, 
    COUNT(g.id_golpe) AS quantidade_golpes
FROM 
    Golpe g
JOIN 
    Tipo t ON g.id_tipo = t.id_tipo
GROUP BY 
    t.nome
ORDER BY 
    quantidade_golpes DESC;
""",
""" 
-- Select 5 - Mostra o nome do pokemon e quais golpes cada pokemon instanciado por um treinador sabe
SELECT 
    DISTINCT(p.nome) AS pokemon, 
    g.nome AS golpe, 
    g.poder
FROM 
    Pokemon p
JOIN 
    Pokemon_eh_de_tal pt 
ON 
	pt.id_pokedex_pokemon = p.id_pokedex_pokemon
JOIN 
    Sabe s 
ON 
	s.id_pokedex_pokemon = p.id_pokedex_pokemon 
           AND s.id_instancia_pokemon = p.id_instancia_pokemon
JOIN 
    Golpe g 
ON 
	g.id_golpe = s.id_golpe
WHERE 
    p.id_instancia_pokemon = 1
ORDER BY 
    p.nome, g.nome;

""",
"""
-- Select 6 - Mostra os pokemons que evoluem com pedra e qual pedra evolui eles (com tratamento pro eevee que evolui com 3 pedras diferentes)
SELECT 
    DISTINCT p.nome AS pokemon, 
    i.nome AS item
FROM 
    Item i
JOIN 
    Item_Evolucao ie ON ie.id_item = i.id_item AND ie.id_instancia = i.id_instancia
JOIN 
    Item_eh_de_tal iet ON iet.id_item = i.id_item AND iet.id_instancia = i.id_instancia
JOIN 
    Pokemon p ON p.evolui_com_pedra = TRUE
WHERE
    (
        p.nome = 'Eevee' AND i.nome IN ('Fire Stone', 'Thunder Stone', 'Water Stone')
    )
    OR
    (
        p.nome != 'Eevee' AND p.id_pokedex_pokemon IN (
            SELECT id_pokedex_pokemon 
            FROM Pokemon_eh_de_tal 
            WHERE id_tipo = iet.id_tipo
        )
    )
ORDER BY 
    i.nome;
""",
"""
-- Select 7 - Mostra em ordem alfabética os pokemons com mais de um tipo
SELECT 
    p.nome, 
    STRING_AGG(DISTINCT t.nome, ', ') AS tipos
FROM 
    Pokemon p
JOIN 
    Pokemon_eh_de_tal pt 
ON 
	pt.id_pokedex_pokemon = p.id_pokedex_pokemon
JOIN 
    Tipo t ON t.id_tipo = pt.id_tipo
GROUP BY 
    p.nome
HAVING 
    COUNT(pt.id_tipo) > 1;
""",
"""
-- Select 8 - Mostra o nome de quais insignas cada treinador tem em ordem de ginásio
SELECT 
    t.nome, 
    STRING_AGG(i.nome, ', ') AS insignias
FROM 
    Treinador t
JOIN 
    Possui p ON p.id_treinador = t.id_treinador
JOIN 
    Insignia i ON i.id_insignia = p.id_insignia
GROUP BY 
    t.nome;
""",
""" 
-- Select 9 -- Mostra a lista de golpes com precisão menor do que 100 de cada pokemon por treinador
SELECT 
    t.nome AS treinador,
    STRING_AGG(g.nome || ' (Poder: ' || g.poder || ', Precisão: ' || g.precisao || ')', ', ') AS golpes
FROM 
    Treinador t
JOIN 
    Equipe e ON e.id_treinador = t.id_treinador
JOIN 
    Eh_composta ec ON ec.id_equipe = e.id_equipe
JOIN 
    Sabe s ON s.id_pokedex_pokemon = ec.id_pokedex_pokemon 
           AND s.id_instancia_pokemon = ec.id_instancia_pokemon
JOIN 
    Golpe g ON g.id_golpe = s.id_golpe
WHERE 
    g.precisao < 100
GROUP BY 
    t.nome
ORDER BY 
    MAX(g.poder) DESC;

""",
"""
-- Select 10 - Mostra os nomes e níveis dos pokemons na equipe dos treinadores que possuem level maior ou igual a 50
SELECT 
    t.nome AS treinador, 
    STRING_AGG(p.nome || ' (Nível: ' || p.nivel || ')', ', ' ORDER BY p.nivel ASC, p.nome ASC) AS pokemon
FROM 
    Treinador t
JOIN 
    Equipe e ON e.id_treinador = t.id_treinador
JOIN 
    Eh_composta ec
ON 
	ec.id_equipe = e.id_equipe
JOIN 
    Pokemon p
ON 
	p.id_pokedex_pokemon = ec.id_pokedex_pokemon 
              AND p.id_instancia_pokemon = ec.id_instancia_pokemon
WHERE 
    p.nivel >= 50
GROUP BY 
    T.nome
ORDER BY 
    MAX(p.nivel) DESC;
""",
"""
-- Select 11 - Mostra o nome de todos os pokemons capturados por cada treinador
SELECT 
    t.nome AS treinador, 
    COUNT(p.id_pokedex_pokemon) AS total_pokemon,
    STRING_AGG(DISTINCT p.nome, ', ') AS pokemons
FROM 
    Treinador t
JOIN 
    Pokemon p 
ON 
	p.id_treinador = T.id_treinador 
GROUP BY 
    t.nome
ORDER BY 
    total_pokemon DESC;

"""
]

# Executando e visualizando os resultados no terminal do Jupyter Notebook
for idx, command in enumerate(sql_commands):
    print(f"\n=== Resultado da Consulta {idx + 1} ===")
    cursor.execute(command)
    col_names = [desc[0] for desc in cursor.description]
    results = cursor.fetchall()
    df = pd.DataFrame(results, columns=col_names)
    display(df)

print("Consultas criadas com sucesso!")


=== Resultado da Consulta 1 ===


,vencedor,vitorias
0,Daniel,6
1,Barbara,3
2,Panza,3
3,Jotave,3
4,Bruno,1
5,Lance,1
6,Agatha,1



=== Resultado da Consulta 2 ===


,treinador,pokemons,ataques,ataques_especiais
0,Jotave,"Charizard, Vileplume","[90, 90]","[110, 110]"
1,Lorelei,"Slowbro, Jynx","[75, 60]","[100, 120]"
2,Lance,"Gyarados, Aerodactyl, Dragonite","[120, 105, 120]","[90, 70, 100]"
3,Panza,"Dragonite, Venusaur, Gyarados","[120, 90, 110]","[100, 110, 80]"
4,Bruno,"Machop, Machoke, Machamp","[100, 110, 120]","[70, 80, 85]"
5,Daniel,"Alakazam, Gyarados, Snorlax, Jolteon, Gengar, ...","[60, 110, 100, 85, 90, 80]","[150, 80, 70, 110, 130, 120]"
6,Barbara,Slowbro,[80],[110]
7,Agatha,"Gengar, Haunter, Gengar","[75, 75, 80]","[130, 120, 140]"



=== Resultado da Consulta 3 ===


...


=== Resultado da Consulta 10 ===


,treinador,pokemon
0,Lance,"Dragonair (Nível: 56), Dragonair (Nível: 56), ..."
1,Bruno,"Onix (Nível: 53), Onix (Nível: 53), Machop (Ní..."
2,Agatha,"Golbat (Nível: 55), Haunter (Nível: 55), Arbok..."
3,Lorelei,"Cloyster (Nível: 54), Slowbro (Nível: 55), Dew..."
4,Panza,"Venusaur (Nível: 50), Dragonite (Nível: 55), G..."
5,Daniel,"Jolteon (Nível: 50), Snorlax (Nível: 50), Star..."
6,Jotave,Charizard (Nível: 50)



=== Resultado da Consulta 11 ===


,treinador,total_pokemon,pokemons
0,Jotave,8,"Butterfree, Charizard, Charmander, Charmeleon,..."
1,Panza,8,"Diglett, Dragonair, Dragonite, Dratini, Gyarad..."
2,Barbara,7,"Chansey, Clefable, Clefairy, Jigglypuff, Licki..."
3,Raquel,6,"Electrode, Growlithe, Koffing, Magmar, Pikachu..."
4,Daniel,6,"Alakazam, Gengar, Gyarados, Jolteon, Snorlax, ..."
5,Lance,5,"Aerodactyl, Dragonair, Dragonite, Gyarados"
6,Lorelei,5,"Cloyster, Dewgong, Jynx, Lapras, Slowbro"
7,Bruno,5,"Machamp, Machoke, Machop, Onix"
8,Agatha,5,"Arbok, Gengar, Golbat, Haunter"


Consultas criadas com sucesso!


In [12]:
# Fechando a conexão inicial
cursor.close()
connection.close()